<a href="https://colab.research.google.com/github/sungkwangsong/EasyOCR/blob/master/notebooks/Hi_BERT_%EB%B0%B0%EC%B9%98_%ED%95%99%EC%8A%B5_(KoSimCSE_%EB%AA%A8%EB%8D%B8_%EB%B2%A0%EC%9D%B4%EC%8A%A4)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
HBN_PROJECT_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT'
HBN_ONTOLOGIES_DIR_PATH = f'{HBN_PROJECT_DIR_PATH}/ontologies'
HBN_OUTPUTS_DIR_PATH = f'{HBN_PROJECT_DIR_PATH}/outputs'
HBN_MODELS_DIR_PATH = f'{HBN_PROJECT_DIR_PATH}/models'
HBN_DATASETS_DIR_PATH = f'{HBN_PROJECT_DIR_PATH}/datasets'
HBN_DATALAKE_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/hbn-data-lake'

In [4]:
import os
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertModel, BertForMaskedLM
# AdamW를 torch.optim에서 가져오도록 수정
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import glob
import random
import pickle
import gc
import time
from torch.amp import autocast, GradScaler
import concurrent.futures
from functools import partial

# 메모리 최적화를 위한 환경 변수 설정
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# 경로 설정
ONTOLOGY_TRAIN_DIR = f"{HBN_ONTOLOGIES_DIR_PATH}/training"
ONTOLOGY_TEST_DIR = f"{HBN_ONTOLOGIES_DIR_PATH}/test"
ONTOLOGY_VALID_DIR = f"{HBN_ONTOLOGIES_DIR_PATH}/validation"
PAPER_TRAIN_PATH = f"{HBN_DATASETS_DIR_PATH}/combined_articles_train.csv"
PAPER_TEST_PATH = f"{HBN_DATASETS_DIR_PATH}/combined_articles_test.csv"
PAPER_VALID_PATH = f"{HBN_DATASETS_DIR_PATH}/combined_articles_validation.csv"
OUTPUT_DIR = f"{HBN_MODELS_DIR_PATH}/hi_bert_model_kosimcse"
STOPWORDS_PATH = f"{HBN_DATASETS_DIR_PATH}/recruitment_stopwords2.csv"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 하이퍼파라미터 설정
MAX_LENGTH = 256
BATCH_SIZE = 32
MLM_LEARNING_RATE = 1e-5 #2e-5
CL_LEARNING_RATE = 5e-6 #1e-5  # Hi-BERT를 위해 낮은 학습률 유지
MLM_EPOCHS = 2 #4
CL_EPOCHS = 3 #4
TEMPERATURE = 0.07 #0.05  # 더 샤프한 대조학습
# MLM_LEARNING_RATE = 2e-5
# CL_LEARNING_RATE = 1e-5  # Hi-BERT를 위해 낮은 학습률 유지
# MLM_EPOCHS = 4
# CL_EPOCHS = 4
# TEMPERATURE = 0.05  # 더 샤프한 대조학습
MARGIN = 0.2  # 마진 손실을 위한 마진 값
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# 그라디언트 누적 스텝 설정
ACCUMULATION_STEPS = 2

MAX_JOB_COUNT = 1000
MAX_PAPER_COUNT = 1000
MAX_ONTOLOGY_COUNT = 1000
MAX_TEMPLATE_COUNT = 500

HBN_DROPOUT = 0.1 # 0.1에서 증가

TEMPLATE_TARGET_MIN = 80
TEMPLATE_TARGET_MAX = 90
TEMPLATE_TARGET_TRY_LIMIT = 20

# 단일 기본 모델 설정 - 일관성을 위해 단일 모델만 사용
# BASE_MODEL = 'klue/bert-base'
BASE_MODEL = 'BM-K/KoSimCSE-bert'
PROJECTION_DIM = 768
IS_RESET_CHECKPOINT = True

# 템플릿 정의
templates = {
    "hasQualification": "{} 직무는 {} 자격증이 필요합니다.",
    "requiresCertificate": "{} 분야는 {} 자격증이 필요합니다.",
    "hasMajor": "{} 분야는 {} 전공과 관련이 있습니다.",
    "requiresMajor": "{} 분야는 {} 전공 지식이 필요합니다.",
    "hasSkill": "{} 직무는 {} 기술이 필요합니다.",
    "requiresSkill": "{} 분야는 {} 기술이 필요합니다.",
    "hasOrganization": "{} 채용은 {} 기관에서 진행합니다.",
    "hasLocation": "{} 기관은 {} 지역에 위치합니다.",
    "hasDepartment": "{} 채용은 {} 부서에서 진행합니다.",
    "hasField": "{} 부서는 {} 분야를 담당합니다.",
    "hasPositionLevel": "{} 분야는 {} 직급을 채용합니다.",
    "hasPreference": "{} 분야는 {} 우대사항이 있습니다.",
    "hasDuty": "{} 분야는 {} 업무를 담당합니다.",
    "hasDuties": "{} 분야는 {} 업무를 담당합니다.",
    "hasMajorKeyword": "{} 전공은 {} 키워드와 관련이 있습니다.",
    "hasMajorKeywords": "{} 전공은 {} 키워드와 관련이 있습니다.",
    "hasRequirement": "{} 분야는 {} 요구사항이 있습니다.",
    "hasRequirements": "{} 분야는 {} 요구사항이 있습니다.",
    "hasRecruitmentField": "{} 채용은 {} 분야를 대상으로 합니다.",
    "hasSkillKnowledge": "{} 직무는 {} 지식/기술이 필요합니다.",
    "hasSkillKnowledges": "{} 직무는 {} 지식/기술이 필요합니다.",
    "hasJobField": "{} 채용은 {} 직무 분야와 관련됩니다.",
    "hasJobPosting": "{} 기관은 {} 채용 공고를 게시했습니다.",
    "hasTitle": "{} 채용의 제목은 {}입니다.",
    "hasDatePosted": "{} 채용 공고는 {}에 게시되었습니다.",
    "hasPreferredConditions": "{} 분야는 {} 우대 조건이 있습니다.",
    "hasPreferredCondition": "{} 분야는 {} 우대 조건이 있습니다.",
    "hasQualifications": "{} 직무는 {} 자격 요건이 필요합니다.",
    "hasMajors": "{} 분야는 {} 전공자를 채용합니다.",
    # 채용-논문 연결에 특화된 추가 템플릿
    "relatedToPaper": "{} 분야는 {} 주제의 논문과 관련이 있습니다.",
    "usesResearch": "{} 직무는 {} 연구 결과를 활용합니다.",
    "requiresKnowledge": "{} 분야는 {} 지식이 필요합니다.",
    "appliesTheory": "{} 직무는 {} 이론을 적용합니다.",
}


# 체크포인트 저장 함수
def save_checkpoint(output_dir, step, model, optimizer, scheduler, tokenizer=None, loss=None):
    checkpoint_dir = f"{output_dir}/checkpoint_{step}"
    os.makedirs(checkpoint_dir, exist_ok=True)

    checkpoint = {
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'loss': loss
    }

    torch.save(checkpoint, f"{checkpoint_dir}/model.pt")

    if tokenizer:
        tokenizer.save_pretrained(checkpoint_dir)

    print(f"Checkpoint saved at step {step}")

    # 체크포인트 정보 저장
    with open(f"{output_dir}/checkpoint_info.json", 'w') as f:
        json.dump({'latest_step': step, 'checkpoint_path': checkpoint_dir}, f)

# 체크포인트 로드 함수
def load_latest_checkpoint(output_dir, model, optimizer=None, scheduler=None):
    checkpoint_info_path = f"{output_dir}/checkpoint_info.json"

    if not os.path.exists(checkpoint_info_path):
        print("No checkpoint found. Starting from scratch.")
        return None, 0

    with open(checkpoint_info_path, 'r') as f:
        checkpoint_info = json.load(f)

    checkpoint_path = checkpoint_info['latest_step']
    checkpoint_dir = checkpoint_info['checkpoint_path']
    checkpoint_file = f"{checkpoint_dir}/model.pt"

    if not os.path.exists(checkpoint_file):
        print(f"Checkpoint file {checkpoint_file} not found. Starting from scratch.")
        return None, 0

    print(f"Loading checkpoint from {checkpoint_file}")
    try:
        checkpoint = torch.load(checkpoint_file, map_location=DEVICE)

        model.load_state_dict(checkpoint['model_state_dict'])

        if optimizer and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        if scheduler and 'scheduler_state_dict' in checkpoint and checkpoint['scheduler_state_dict']:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        # 스텝 값 처리 - 문자열이면 숫자로 변환 시도
        step = checkpoint['step']
        if isinstance(step, str):
            # 에포크 형식인 경우(예: 'epoch_2') 특별 처리
            if step.startswith('epoch_'):
                # MLM 학습 완료로 간주
                print(f"Detected epoch checkpoint: {step}. MLM training is considered complete.")
                return checkpoint, 999999  # 매우 큰 숫자로 설정하여 학습 완료로 처리
            else:
                # 다른 형태의 문자열이면 정수로 변환 시도
                try:
                    step = int(step)
                except ValueError:
                    print(f"Could not convert step '{step}' to integer. Starting from step 0.")
                    step = 0

        print(f"Checkpoint loaded. Resuming from step {step}")
        return checkpoint, step

    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None, 0

# 불용어 로드 함수
def load_stopwords(file_path):
    """불용어 CSV 파일 로드"""
    try:
        stopwords_df = pd.read_csv(file_path)

        # 한국어, 약어, 영어 불용어 모두 수집
        stopwords = set()

        # 한국어 불용어
        if 'KOR' in stopwords_df.columns:
            kor_stopwords = [word.lower() for word in stopwords_df['KOR'].dropna() if isinstance(word, str)]
            stopwords.update(kor_stopwords)

        # 약어 불용어
        if 'ABB' in stopwords_df.columns:
            abb_stopwords = [word.lower() for word in stopwords_df['ABB'].dropna() if isinstance(word, str)]
            stopwords.update(abb_stopwords)

        # 영어 불용어
        if 'ENG' in stopwords_df.columns:
            eng_stopwords = [word.lower() for word in stopwords_df['ENG'].dropna() if isinstance(word, str)]
            stopwords.update(eng_stopwords)

        # 기본 불용어도 추가
        basic_stopwords = {'및', '등', '를', '이', '를 위한', '기반', '연구', '기술', '분석', '개발', '관한', '통한', '활용',
                          '에서', '으로', '에', '의', '이해', '관련', '중심', '방법', '있는', '위한', '대한', '시스템'}
        stopwords.update(basic_stopwords)

        print(f"Loaded {len(stopwords)} stopwords from {file_path}")
        return stopwords
    except Exception as e:
        print(f"Error loading stopwords file {file_path}: {e}")
        # 실패시 기본 불용어 사용
        return {'및', '등', '를', '이', '를 위한', '기반', '연구', '기술', '분석', '개발', '관한', '통한', '활용',
               '에서', '으로', '에', '의', '이해', '관련', '중심', '방법', '있는', '위한', '대한', '시스템'}

STOPWORDS = load_stopwords(STOPWORDS_PATH)

# 온톨로지 데이터에서 분야 정보 추출 함수
def extract_field_info_from_ontology(ontology_data):
    """온톨로지 데이터에서 분야 정보 추출 - has 접두사 추가된 키 구조에 맞게 수정"""
    field_info = {
        'field': '',
        'duties': '',
        'qualifications': '',
        'preferred': '',
        'majors': '',
        'keywords': '',
        'skills': '',
        'majors_list': [],
        'keywords_list': [],
        'skills_list': []
    }

    # JobPosting 정보
    job_posting = ontology_data.get('hasJobPosting', {})
    if isinstance(job_posting, dict):
        # 이름이 hasTitle로 변경됨
        field_info['field'] = job_posting.get('hasTitle', '')

    # hasJobFields 처리
    if 'hasJobFields' in ontology_data and isinstance(ontology_data['hasJobFields'], list):
        for job_field in ontology_data['hasJobFields']:
            if not isinstance(job_field, dict) or 'hasFields' not in job_field:
                continue

            for field in job_field['hasFields']:
                if not isinstance(field, dict):
                    continue

                # hasRecruitmentField (수정됨)
                if 'hasRecruitmentField' in field:
                    field_info['field'] = field['hasRecruitmentField']

                # hasDuties (수정됨)
                duties = []
                if 'hasDuties' in field and isinstance(field['hasDuties'], list):
                    for duty in field['hasDuties']:
                        if isinstance(duty, dict) and 'hasDescription' in duty:  # 수정됨
                            duties.append(duty['hasDescription'])
                field_info['duties'] = ' '.join(duties)

                # hasQualifications (수정됨)
                quals = []
                if 'hasQualifications' in field and isinstance(field['hasQualifications'], list):
                    for qual in field['hasQualifications']:
                        if isinstance(qual, dict) and 'hasRequirement' in qual:  # 수정됨
                            quals.append(qual['hasRequirement'])
                field_info['qualifications'] = ' '.join(quals)

                # hasPreferredConditions (수정됨)
                prefs = []
                if 'hasPreferredConditions' in field and isinstance(field['hasPreferredConditions'], list):
                    for pref in field['hasPreferredConditions']:
                        if isinstance(pref, dict) and 'hasCondition' in pref:  # 수정됨
                            prefs.append(pref['hasCondition'])
                field_info['preferred'] = ' '.join(prefs)

                # hasMajors (수정됨)
                majors = []
                if 'hasMajors' in field and isinstance(field['hasMajors'], list):
                    for major in field['hasMajors']:
                        if isinstance(major, dict) and 'hasName' in major:  # 수정됨
                            majors.append(major['hasName'])
                field_info['majors'] = ' '.join(majors)
                field_info['majors_list'] = majors

                # hasMajorKeywords (수정됨)
                keywords = []
                if 'hasMajorKeywords' in field and isinstance(field['hasMajorKeywords'], list):
                    for keyword in field['hasMajorKeywords']:
                        if isinstance(keyword, dict) and 'hasKeyword' in keyword:  # 수정됨
                            keywords.append(keyword['hasKeyword'])
                field_info['keywords'] = ' '.join(keywords)
                field_info['keywords_list'] = keywords

                # hasSkillKnowledges (수정됨)
                skills = []
                if 'hasSkillKnowledges' in field and isinstance(field['hasSkillKnowledges'], list):
                    for skill in field['hasSkillKnowledges']:
                        if isinstance(skill, dict) and 'hasSkill' in skill:  # 수정됨
                            skills.append(skill['hasSkill'])
                field_info['skills'] = ' '.join(skills)
                field_info['skills_list'] = skills

    return field_info

# 온톨로지 파일 로드 함수
def load_ontology(directory_path, max_samples=500):
    """온톨로지 파일을 로드하여 채용 정보 추출"""
    job_fields = []

    # MAX_ONTOLOGY_COUNT 적용
    if max_samples is None and MAX_ONTOLOGY_COUNT is not None:
        max_samples = MAX_ONTOLOGY_COUNT

    # 디렉토리 내의 모든 jsonld 파일 찾기
    jsonld_files = glob.glob(os.path.join(directory_path, "**", "*.json"), recursive=True)

    # 파일 수 제한
    if max_samples is not None and max_samples > 0:
        jsonld_files = jsonld_files[:max_samples]

    print(f"Found {len(jsonld_files)} ontology files in {directory_path}")

    # 첫 번째 파일 디버깅 출력
    if jsonld_files:
        first_file = jsonld_files[0]
        print(f"디버깅: 첫 번째 파일 경로 = {first_file}")
        try:
            with open(first_file, 'r', encoding='utf-8') as f:
                try:
                    content = f.read()
                    print(f"파일 내용 (처음 500자):\n{content[:500]}")
                    first_ontology = json.loads(content)
                    print("JSON 구조:")
                    print(f"주요 키: {list(first_ontology.keys())}")
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류: {e}")
                except Exception as e:
                    print(f"파일 처리 오류: {e}")
        except Exception as e:
            print(f"파일 열기 오류: {e}")

    # 파일 반복 처리
    for file_path in tqdm(jsonld_files, desc="Loading ontology files"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    content = f.read()
                    ontology_data = json.loads(content)

                    # 필드 정보 추출
                    field_info = extract_field_info_from_ontology(ontology_data)

                    # 내용 구성 및 결과 추가
                    if field_info:
                        # 채용 정보 텍스트 구성
                        job_text = f"분야: {field_info['field']} 업무: {field_info['duties']} 자격요건: {field_info['qualifications']} " \
                                 f"우대사항: {field_info['preferred']} 전공: {field_info['majors']} " \
                                 f"키워드: {field_info['keywords']} 기술: {field_info['skills']}"

                        job_fields.append({
                            'job_id': ontology_data.get('@id', ''),
                            'job_text': job_text,
                            'field': field_info['field'],
                            'duties': field_info['duties'],
                            'qualifications': field_info['qualifications'],
                            'preferred': field_info['preferred'],
                            'majors': field_info['majors_list'],
                            'keywords': field_info['keywords_list'],
                            'skills': field_info['skills_list']
                        })
                except json.JSONDecodeError as e:
                    print(f"Error parsing {file_path}: {e}")
                except Exception as e:
                    print(f"Error processing {file_path}: {str(e)}")
        except Exception as e:
            print(f"Error opening {file_path}: {e}")

    # 결과 출력
    if len(job_fields) > 0:
        print("\n처리된 첫 번째 채용공고 정보:")
        print(f"ID: {job_fields[0]['job_id']}")
        print(f"분야: {job_fields[0]['field']}")
        print(f"키워드: {job_fields[0]['keywords']}")
    else:
        print("처리된 채용공고가 없습니다!")

    print(f"Successfully loaded {len(job_fields)} job fields from {len(jsonld_files)} files")
    return job_fields

# 논문 데이터 로드 함수
def load_papers(file_path, max_samples=2000):
    """논문 CSV 파일 로드"""
    # MAX_PAPER_COUNT 적용
    if max_samples is None and MAX_PAPER_COUNT is not None:
        max_samples = MAX_PAPER_COUNT

    papers_df = pd.read_csv(file_path)

    # 논문 수 제한
    if max_samples is not None and max_samples > 0:
        papers_df = papers_df.head(max_samples)

    print(f"Processing {len(papers_df)} papers from {file_path}")

    papers = []
    for _, row in tqdm(papers_df.iterrows(), total=len(papers_df), desc="Loading papers"):
        try:
            # 논문 정보 텍스트 구성
            paper_text = f"제목: {row['title']} 저자: {row['authors']} 초록: {row['abstract']} 키워드: {row['keywords']}"

            # 키워드 처리 (불용어 제거 및 길이 2 미만 제외)
            keywords = []
            if isinstance(row['keywords'], str):
                keywords = [k.strip() for k in row['keywords'].split(',')
                           if k.strip().lower() not in STOPWORDS and len(k.strip()) > 2]

            # 카테고리 정보
            category = row.get('category', "") if pd.notna(row.get('category', "")) else ""

            papers.append({
                'paper_id': row['article_id'],
                'paper_text': paper_text,
                'title': row['title'] if pd.notna(row['title']) else "",
                'abstract': row['abstract'] if pd.notna(row['abstract']) else "",
                'keywords': keywords,
                'authors': row['authors'] if pd.notna(row['authors']) else "",
                'category': category,
                'journal': row.get('journal_name', '') if pd.notna(row.get('journal_name', '')) else "",
                'pub_year': row.get('pub_year', '') if pd.notna(row.get('pub_year', '')) else ""
            })
        except Exception as e:
            print(f"Error processing paper row: {e}")
            continue

    print(f"Successfully loaded {len(papers)} papers")
    return papers

# 템플릿 기반 문장 생성 함수
# def generate_template_sentences(job_fields, papers, templates):
#     """온톨로지 템플릿을 활용하여 학습용 문장 생성 (수정된 온톨로지 구조 지원)"""
#     print("\n=== 템플릿 기반 문장 생성 시작 ===")
#     template_sentences = []

#     # 1. 채용 필드 기반 템플릿 문장 생성
#     for job in tqdm(job_fields, desc="Generating job field templates"):
#         field_name = job.get('field', '')
#         if not field_name:
#             continue

#         # 전공 관련 문장 생성
#         for major in job.get('majors', []):  # [:3]:  # 최적화: 최대 3개만 사용
#             if major and len(major) > 1:
#                 template_sentences.append(templates['hasMajor'].format(field_name, major))
#                 template_sentences.append(templates['requiresMajor'].format(field_name, major))

#         # 키워드 관련 문장 생성
#         for keyword in job.get('keywords', []):  # [:3]:  # 최적화: 최대 3개만 사용
#             if keyword and len(keyword) > 1:
#                 template_sentences.append(templates['hasMajorKeyword'].format(field_name, keyword))

#     # 2. 채용-논문 연결을 위한 추가 템플릿 문장 생성
#     # 모든 직무와 논문 사용하여 더 많은 연결 생성 시도
#     job_sample = job_fields[:min(300, len(job_fields))]
#     paper_sample = papers[:min(500, len(papers))]

#     for job in tqdm(job_sample, desc="Generating job-paper templates"):
#         field_name = job.get('field', '')
#         if not field_name:
#             continue

#         # 직무 키워드와 관련된 논문 찾기
#         job_keywords = set([k.lower() for k in job.get('keywords', []) if k])
#         job_skills = set([s.lower() for s in job.get('skills', []) if s])

#         # 키워드나 스킬이 없는 경우, 직무명을 토큰화하여 키워드로 사용
#         if not job_keywords and field_name:
#             job_keywords = set([w.lower() for w in field_name.split() if len(w) > 2 and w.lower() not in STOPWORDS])

#         for paper in paper_sample[:20]:  # 각 직무당 최대 20개 논문 확인
#             paper_keywords = set([k.lower() for k in paper.get('keywords', []) if k])
#             paper_title = paper.get('title', '')

#             # 논문 키워드가 없는 경우, 제목을 토큰화하여 키워드로 사용
#             if not paper_keywords and paper_title:
#                 paper_keywords = set([w.lower() for w in paper_title.split() if len(w) > 2 and w.lower() not in STOPWORDS])

#             # 키워드 일치 여부 확인 (부분 일치도 포함)
#             keyword_match = False
#             for job_kw in job_keywords:
#                 for paper_kw in paper_keywords:
#                     if job_kw in paper_kw or paper_kw in job_kw:
#                         matched_keyword = job_kw if len(job_kw) > len(paper_kw) else paper_kw
#                         template_sentences.append(templates['relatedToPaper'].format(field_name, matched_keyword))
#                         keyword_match = True
#                         break
#                 if keyword_match:
#                     break

#             # 키워드 매칭이 없어도 일부 조합은 생성
#             if len(paper_title) > 5 and len(field_name) > 3:
#                 # 10% 확률로 임의 조합 생성
#                 if random.random() < 0.1:
#                     template_sentences.append(templates['usesResearch'].format(field_name, paper_title))

#                     # 수정된 부분: 빈 리스트 체크 추가
#                     paper_keywords_list = paper.get('keywords', [])
#                     if paper_keywords_list:
#                         default_keyword = paper_keywords_list[0]
#                     else:
#                         default_keyword = '연구'
#                     template_sentences.append(templates['requiresKnowledge'].format(field_name, default_keyword))

#     # 중복 제거
#     template_sentences = list(set(template_sentences))

#     # 템플릿 문장이 너무 적으면 기본 템플릿 추가
#     if len(template_sentences) < 100:
#         print("생성된 템플릿 문장이 너무 적어 기본 템플릿을 추가합니다.")

#         # 직무와 스킬 조합으로 기본 템플릿 추가
#         for job in job_sample[:50]:
#             field_name = job.get('field', '')
#             if not field_name:
#                 continue

#             # 기본 템플릿 추가
#             template_sentences.append(f"{field_name} 분야는 전문 지식이 필요합니다.")
#             template_sentences.append(f"{field_name} 분야는 연구 결과를 활용합니다.")

#             # 스킬 기반 템플릿 추가
#             for skill in job.get('skills', []):
#                 if skill:
#                     template_sentences.append(f"{field_name} 분야는 {skill} 기술이 필요합니다.")

#             # 전공 기반 템플릿 추가
#             for major in job.get('majors', []):
#                 if major:
#                     template_sentences.append(f"{field_name} 분야는 {major} 전공과 관련이 있습니다.")

#     # 최대 5000개 문장만 사용 (최적화)
#     if len(template_sentences) > MAX_TEMPLATE_COUNT:
#         template_sentences = random.sample(template_sentences, MAX_TEMPLATE_COUNT)
#     elif len(template_sentences) == 0:
#         # 템플릿 문장이 하나도 없으면 최소한의 문장 생성
#         print("템플릿 문장이 생성되지 않아 기본 문장을 추가합니다.")
#         for i in range(100):
#             template_sentences.append(f"채용 분야는 관련 기술이 필요합니다_{i}.")
#             template_sentences.append(f"연구 분야는 관련 지식이 필요합니다_{i}.")

#     print(f"생성된 고유 템플릿 문장 수: {len(template_sentences)}")

#     return template_sentences

def generate_template_sentences(job_fields, papers, templates):
    """온톨로지 템플릿을 활용하여 학습용 문장 생성 (수정된 온톨로지 구조 지원)"""
    print("\n=== 템플릿 기반 문장 생성 시작 ===")
    template_sentences = []

    # 1. 채용 필드 기반 템플릿 문장 생성
    for job in tqdm(job_fields, desc="Generating job field templates"):
        field_name = job.get('field', '')
        if not field_name:
            continue

        # 전공 관련 문장 생성
        for major in job.get('majors', []): #[:3]:  # 최적화: 최대 3개만 사용
            if major and len(major) > 1:
                template_sentences.append(templates['hasMajor'].format(field_name, major))
                template_sentences.append(templates['requiresMajor'].format(field_name, major))

        # 기술/스킬 관련 문장 생성
        for skill in job.get('skills', [])[:3]:  # 최적화: 최대 3개만 사용
            if skill and len(skill) > 1:
                template_sentences.append(templates['hasSkill'].format(field_name, skill))
                template_sentences.append(templates['requiresSkill'].format(field_name, skill))

        # 키워드 관련 문장 생성
        for keyword in job.get('keywords', []): #[:3]:  # 최적화: 최대 3개만 사용
            if keyword and len(keyword) > 1:
                template_sentences.append(templates['hasMajorKeyword'].format(field_name, keyword))

        # 업무 관련 문장 생성
        if job.get('duties'):
            duties = [d.strip() for d in job['duties'].split('.') if d.strip()]
            for duty in duties[:2]:  # 최적화: 최대 2개 업무만 사용
                if duty and len(duty) > 5:  # 너무 짧은 문장 제외, 길이 제한 조정
                    template_sentences.append(templates['hasDuty'].format(field_name, duty))

        # 자격 요건 관련 문장 생성
        if job.get('qualifications'):
            quals = [q.strip() for q in job['qualifications'].split('.') if q.strip()]
            for qual in quals[:2]:  # 최적화: 최대 2개 자격요건만 사용
                if qual and len(qual) > 5:  # 길이 제한 조정
                    template_sentences.append(templates['hasRequirement'].format(field_name, qual))

        # 우대사항 관련 문장 생성
        if job.get('preferred'):
            prefs = [p.strip() for p in job['preferred'].split('.') if p.strip()]
            for pref in prefs[:1]:  # 최적화: 최대 1개 우대사항만 사용
                if pref and len(pref) > 5:  # 길이 제한 조정
                    template_sentences.append(templates['hasPreference'].format(field_name, pref))

    # 2. 채용-논문 연결을 위한 추가 템플릿 문장 생성
    # 모든 직무와 논문 사용하여 더 많은 연결 생성 시도
    job_sample = job_fields[:min(300, len(job_fields))]
    paper_sample = papers[:min(500, len(papers))]

    for job in tqdm(job_sample, desc="Generating job-paper templates"):
        field_name = job.get('field', '')
        if not field_name:
            continue

        # 직무 키워드와 관련된 논문 찾기
        job_keywords = set([k.lower() for k in job.get('keywords', []) if k])
        job_skills = set([s.lower() for s in job.get('skills', []) if s])

        # 키워드나 스킬이 없는 경우, 직무명을 토큰화하여 키워드로 사용
        if not job_keywords and field_name:
            job_keywords = set([w.lower() for w in field_name.split() if len(w) > 2 and w.lower() not in STOPWORDS])

        for paper in paper_sample[:20]:  # 각 직무당 최대 20개 논문 확인
            paper_keywords = set([k.lower() for k in paper.get('keywords', []) if k])
            paper_title = paper.get('title', '')

            # 논문 키워드가 없는 경우, 제목을 토큰화하여 키워드로 사용
            if not paper_keywords and paper_title:
                paper_keywords = set([w.lower() for w in paper_title.split() if len(w) > 2 and w.lower() not in STOPWORDS])

            # 키워드 일치 여부 확인 (부분 일치도 포함)
            keyword_match = False
            for job_kw in job_keywords:
                for paper_kw in paper_keywords:
                    if job_kw in paper_kw or paper_kw in job_kw:
                        matched_keyword = job_kw if len(job_kw) > len(paper_kw) else paper_kw
                        template_sentences.append(templates['relatedToPaper'].format(field_name, matched_keyword))
                        keyword_match = True
                        break
                if keyword_match:
                    break

            # 키워드 매칭이 없어도 일부 조합은 생성
            if len(paper_title) > 5 and len(field_name) > 3:
                # 10% 확률로 임의 조합 생성
                if random.random() < 0.1:
                    template_sentences.append(templates['usesResearch'].format(field_name, paper_title))
                    template_sentences.append(templates['requiresKnowledge'].format(field_name, paper.get('keywords', ['연구'])[0]))

    # 중복 제거
    template_sentences = list(set(template_sentences))

    # 템플릿 문장이 너무 적으면 기본 템플릿 추가
    if len(template_sentences) < 100:
        print("생성된 템플릿 문장이 너무 적어 기본 템플릿을 추가합니다.")

        # 직무와 스킬 조합으로 기본 템플릿 추가
        for job in job_sample[:50]:
            field_name = job.get('field', '')
            if not field_name:
                continue

            # 기본 템플릿 추가
            template_sentences.append(f"{field_name} 분야는 전문 지식이 필요합니다.")
            template_sentences.append(f"{field_name} 분야는 연구 결과를 활용합니다.")

            # 스킬 기반 템플릿 추가
            for skill in job.get('skills', []):
                if skill:
                    template_sentences.append(f"{field_name} 분야는 {skill} 기술이 필요합니다.")

            # 전공 기반 템플릿 추가
            for major in job.get('majors', []):
                if major:
                    template_sentences.append(f"{field_name} 분야는 {major} 전공과 관련이 있습니다.")

    # 최대 5000개 문장만 사용 (최적화)
    if len(template_sentences) > MAX_TEMPLATE_COUNT:
        template_sentences = random.sample(template_sentences, MAX_TEMPLATE_COUNT)
    elif len(template_sentences) == 0:
        # 템플릿 문장이 하나도 없으면 최소한의 문장 생성
        print("템플릿 문장이 생성되지 않아 기본 문장을 추가합니다.")
        for i in range(100):
            template_sentences.append(f"채용 분야는 관련 기술이 필요합니다_{i}.")
            template_sentences.append(f"연구 분야는 관련 지식이 필요합니다_{i}.")

    print(f"생성된 고유 템플릿 문장 수: {len(template_sentences)}")

    return template_sentences

# 1. MLM 학습을 위한 데이터셋 클래스
class DomainTextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze()
        }

# 2. 대조학습을 위한 데이터셋 클래스
class ContrastivePairDataset(Dataset):
    def __init__(self, job_papers_pairs, tokenizer, max_length):
        self.pairs = job_papers_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]
        job = pair['job']
        paper = pair['paper']

        job_text = job['job_text']
        paper_text = paper['paper_text']

        job_encoding = self.tokenizer(
            job_text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        paper_encoding = self.tokenizer(
            paper_text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'job_input_ids': job_encoding['input_ids'].squeeze(),
            'job_attention_mask': job_encoding['attention_mask'].squeeze(),
            'paper_input_ids': paper_encoding['input_ids'].squeeze(),
            'paper_attention_mask': paper_encoding['attention_mask'].squeeze(),
            'score': torch.tensor(pair['score'], dtype=torch.float)
        }


def generate_templates_with_target_success_rate(job_fields, papers, templates,
                                               target_min=80, target_max=90,
                                               max_attempts=20,
                                               threshold_step=0.05):
    """목표 성공률에 도달할 때까지 템플릿 생성 반복 시도"""

    current_threshold = 0.2  # 초기 임계값 설정

    for attempt in range(max_attempts):
        print(f"\n시도 {attempt+1}/{max_attempts}: 템플릿 생성 중... (임계값: {current_threshold:.2f})")

        # 템플릿 생성 (적절한 랜덤성 추가)
        random_seed = 42 + attempt  # 매 시도마다 다른 시드 사용 (SAMPLE_SEED 대신 고정값 사용)
        random.seed(random_seed)

        # 템플릿 생성
        template_sentences = generate_template_sentences(job_fields, papers, templates)

        # 템플릿으로 관계 추출 (임계값 조정)
        template_relations, matching_stats = extract_relations_from_templates(
            template_sentences,
            job_fields,
            papers,
            tokenizer=None,
            model=None,
            threshold=current_threshold,  # 현재 임계값 사용
            device=DEVICE
        )

        # 성공률 계산
        total_attempts = matching_stats['total_matches']
        successful_matches = matching_stats['successful_matches']

        if total_attempts > 0:
            success_rate = (successful_matches / total_attempts) * 100
        else:
            success_rate = 0

        print(f"템플릿 매칭 결과: 시도 {total_attempts}, 성공 {successful_matches}, 성공률 {success_rate:.2f}%")

        # 목표 성공률 범위 내에 있는지 확인
        if target_min <= success_rate <= target_max:
            print(f"목표 성공률 달성! ({success_rate:.2f}%)")
            return template_sentences, template_relations, matching_stats

        # 임계값 조정
        if success_rate < target_min:
            # 성공률이 너무 낮으면 임계값 낮추기
            current_threshold -= threshold_step
            print(f"성공률이 너무 낮습니다. 임계값을 {current_threshold:.2f}로 낮춥니다.")
        else:  # success_rate > target_max
            # 성공률이 너무 높으면 임계값 높이기
            current_threshold += threshold_step
            print(f"성공률이 너무 높습니다. 임계값을 {current_threshold:.2f}로 높입니다.")

        # 임계값 범위 제한 (0.05~0.95)
        current_threshold = max(0.05, min(0.95, current_threshold))

    # 최대 시도 횟수를 초과해도 목표 성공률에 도달하지 못한 경우
    print(f"최대 시도 횟수({max_attempts})에 도달했으나 목표 성공률에 도달하지 못했습니다.")
    print(f"마지막 시도의 성공률({success_rate:.2f}%)로 진행합니다.")

    return template_sentences, template_relations, matching_stats

# 관계 추출 함수
# 최적화된 관계 추출 함수 (배치 처리 및 병렬 처리 적용)
# extract_relations_from_templates 함수 수정 - 매칭 통계 추가
def extract_relations_from_templates(template_sentences, job_fields, papers, tokenizer=None, model=None, threshold=0.2, device=DEVICE):
    """템플릿 문장에서 채용공고-논문 관계 추출 (배치 및 병렬 처리 최적화)"""
    relations = {}
    matching_stats = {'total_matches': 0, 'successful_matches': 0}  # 매칭 통계 추가

    # 토큰화 및 불용어 제거 함수
    def tokenize(text):
        if not isinstance(text, str):
            return []
        return [w.lower() for w in text.split() if w.lower() not in STOPWORDS and len(w) > 2]

    print("템플릿 문장에서 채용공고-논문 관계 추출 중...")

    # 디버깅: 첫 10개 템플릿 문장 출력
    print("===== 템플릿 문장 샘플 =====")
    for i, sentence in enumerate(template_sentences[:10]):
        print(f"템플릿 문장 {i+1}: {sentence}")
    print("===========================")

    # 디버깅: 첫 5개 채용공고 정보 출력
    print("===== 채용공고 샘플 =====")
    for i, job in enumerate(job_fields[:5]):
        job_id = job.get('job_id', 'ID 없음')
        field = job.get('field', '분야 없음')
        keywords = job.get('keywords', [])
        skills = job.get('skills', [])
        print(f"채용공고 {i+1} (ID: {job_id}): 분야={field}, 키워드={keywords}, 기술={skills}")
    print("===========================")

    # 디버깅: 첫 5개 논문 정보 출력
    print("===== 논문 샘플 =====")
    for i, paper in enumerate(papers[:5]):
        paper_id = paper.get('paper_id', 'ID 없음')
        title = paper.get('title', '제목 없음')
        keywords = paper.get('keywords', [])
        print(f"논문 {i+1} (ID: {paper_id}): 제목={title}, 키워드={keywords}")
    print("===========================")

    # 샘플 크기 감소 (처리 속도 개선)
    job_sample = job_fields[:30]  # 50개에서 30개로 감소
    paper_sample = papers[:50]    # 100개에서 50개로 감소

    # 논문 및 채용공고 전처리 (병렬 처리)
    def preprocess_job(job):
        job_id = job.get('job_id', '')
        field_name = job.get('field', '')
        job_keywords = set([k.lower() for k in job.get('keywords', []) if k])
        job_skills = set([s.lower() for s in job.get('skills', []) if s])

        # 키워드가 없으면 필드명에서 추출
        if not job_keywords and field_name:
            job_keywords = set([w.lower() for w in field_name.split()
                              if len(w) > 2 and w.lower() not in STOPWORDS])

        return {
            'job_id': job_id,
            'field_name': field_name,
            'job_keywords': job_keywords,
            'job_skills': job_skills
        }

    def preprocess_paper(paper):
        paper_id = paper.get('paper_id', '')
        paper_title = paper.get('title', '')
        paper_keywords = set([k.lower() for k in paper.get('keywords', []) if k])

        # 키워드가 없으면 제목에서 추출
        if not paper_keywords and paper_title:
            paper_keywords = set([w.lower() for w in paper_title.split()
                               if len(w) > 2 and w.lower() not in STOPWORDS])

        title_tokens = tokenize(paper_title) if paper_title else []

        return {
            'paper_id': paper_id,
            'paper_title': paper_title,
            'paper_keywords': paper_keywords,
            'title_tokens': title_tokens
        }

    # 병렬 전처리 실행
    print("채용공고 및 논문 데이터 전처리 중...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        processed_jobs = list(executor.map(preprocess_job, job_sample))
        processed_papers = list(executor.map(preprocess_paper, paper_sample))

    print(f"전처리 완료: {len(processed_jobs)} 채용공고, {len(processed_papers)} 논문")

    # 템플릿 문장 토큰화 (병렬 처리)
    def process_template(sentence):
        return {'text': sentence, 'tokens': tokenize(sentence)}

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        processed_templates = list(executor.map(process_template, template_sentences))

    # 배치 매칭 함수 (여러 채용공고-논문 쌍을 한 번에 처리)
    def process_template_batch(template_batch, jobs, papers):
        local_relations = {}
        match_attempts = 0
        match_success = 0

        for template in template_batch:
            sentence = template['text']
            sentence_tokens = template['tokens']

            for job in jobs:
                job_id = job['job_id']
                field_name = job['field_name']

                # 필드명이 템플릿에 없으면 스킵 (빠른 필터링)
                if not field_name or field_name.lower() not in sentence.lower():
                    continue

                job_keywords = job['job_keywords']

                # 채용공고 키워드와 템플릿 문장의 일치도 확인
                job_relevance = 0
                if job_keywords:
                    common_tokens = job_keywords.intersection(set(sentence_tokens))
                    if common_tokens:
                        job_relevance = len(common_tokens) / len(job_keywords)

                # 관련성이 없으면 스킵 (빠른 필터링)
                if job_relevance == 0:
                    continue

                for paper in papers:
                    match_attempts += 1
                    paper_id = paper['paper_id']
                    paper_keywords = paper['paper_keywords']
                    title_tokens = paper['title_tokens']

                    # 논문 키워드와 템플릿 문장의 일치도 확인
                    paper_relevance = 0
                    if paper_keywords:
                        common_tokens = paper_keywords.intersection(set(sentence_tokens))
                        if common_tokens:
                            paper_relevance = len(common_tokens) / len(paper_keywords)

                    # 논문 제목의 단어가 템플릿 문장에 포함되는지 확인
                    title_relevance = 0
                    if title_tokens:
                        common_title_tokens = set(title_tokens).intersection(set(sentence_tokens))
                        if common_title_tokens:
                            title_relevance = len(common_title_tokens) / len(title_tokens)

                    # 채용공고-논문 키워드 직접 매칭
                    keyword_match = 0
                    if job_keywords and paper_keywords:
                        common_keywords = job_keywords.intersection(paper_keywords)
                        if common_keywords:
                            keyword_match = len(common_keywords) / min(len(job_keywords), len(paper_keywords))

                    # 부분 키워드 매칭 (최적화 버전)
                    partial_match = 0
                    if job_keywords and paper_keywords:
                        for j_kw in job_keywords:
                            if partial_match > 0:
                                break
                            if len(j_kw) <= 2:
                                continue
                            for p_kw in paper_keywords:
                                if len(p_kw) <= 2:
                                    continue
                                if j_kw in p_kw or p_kw in j_kw:
                                    partial_match = 0.5
                                    break

                    # 총 매칭 점수 계산
                    match_score = (job_relevance * 0.3 +
                                 paper_relevance * 0.3 +
                                 title_relevance * 0.2 +
                                 keyword_match * 0.3 +
                                 partial_match * 0.2)

                    # 임계값 이상인 경우 관계 추가
                    if match_score > threshold:  # 파라미터로 받은 임계값 사용
                        match_success += 1
                        key = (job_id, paper_id)
                        if key not in local_relations:
                            local_relations[key] = 0
                        local_relations[key] += match_score

        return local_relations, match_attempts, match_success

    # 템플릿 문장을 배치로 나누어 병렬 처리
    batch_size = 100  # 배치당 템플릿 문장 수
    template_batches = [processed_templates[i:i+batch_size]
                        for i in range(0, len(processed_templates), batch_size)]

    total_match_attempts = 0
    total_match_success = 0

    print(f"템플릿 문장 처리 중... (총 {len(template_batches)} 배치)")

    # 병렬 처리를 위한 함수
    process_func = partial(process_template_batch,
                          jobs=processed_jobs,
                          papers=processed_papers)

    # 병렬 배치 처리 실행
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(tqdm(executor.map(process_func, template_batches),
                           total=len(template_batches),
                           desc="템플릿 배치 처리"))

    # 결과 합치기
    for batch_relations, attempts, success in results:
        total_match_attempts += attempts
        total_match_success += success

        # 관계 병합
        for key, score in batch_relations.items():
            if key not in relations:
                relations[key] = 0
            relations[key] += score

    # 디버깅: 매칭 통계 출력
    success_rate = (total_match_success/max(1, total_match_attempts))*100
    print(f"\n총 매칭 시도: {total_match_attempts}, 성공: {total_match_success}, 성공률: {success_rate:.2f}%")

    # 매칭 통계 저장
    matching_stats['total_matches'] = total_match_attempts
    matching_stats['successful_matches'] = total_match_success

    # 관계가 너무 적으면 기본 관계 생성
    if len(relations) < 100:
        print("매칭된 관계가 적어 기본 관계를 생성합니다.")

        # 채용공고-논문 직접 키워드 매칭으로 기본 관계 생성
        def generate_basic_relations(job_batch, papers):
            local_relations = {}

            for job in job_batch:
                job_id = job['job_id']
                job_keywords = job['job_keywords']
                job_field = job['field_name']

                if not job_id or not job_keywords:
                    continue

                for paper in papers:
                    paper_id = paper['paper_id']
                    paper_keywords = paper['paper_keywords']

                    if not paper_id or not paper_keywords:
                        continue

                    # 키워드 교집합으로 관계 생성
                    common_keywords = job_keywords.intersection(paper_keywords)

                    # 부분 매칭 확인
                    has_partial_match = False
                    for j_kw in job_keywords:
                        if has_partial_match:
                            break
                        if len(j_kw) <= 2:
                            continue
                        for p_kw in paper_keywords:
                            if len(p_kw) <= 2:
                                continue
                            if j_kw in p_kw or p_kw in j_kw:
                                has_partial_match = True
                                break

                    # 관계 추가
                    if common_keywords or has_partial_match:
                        key = (job_id, paper_id)
                        local_relations[key] = len(common_keywords) * 0.5 + (0.3 if has_partial_match else 0)

            return local_relations

        # 채용공고를 배치로 나누어 병렬 처리
        job_batches = [processed_jobs[i:i+10] for i in range(0, len(processed_jobs), 10)]

        # 병렬 처리
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            basic_relations_list = list(executor.map(
                lambda job_batch: generate_basic_relations(job_batch, processed_papers),
                job_batches))

        # 기본 관계 병합
        for basic_relations in basic_relations_list:
            for key, score in basic_relations.items():
                if key not in relations:
                    relations[key] = 0
                relations[key] += score

        print(f"기본 관계를 추가하여 총 {len(relations)}개의 관계가 생성되었습니다.")

    print(f"템플릿 문장에서 {len(relations)}개의 관계를 추출했습니다.")
    return relations, matching_stats  # 수정: 관계와 매칭 통계 함께 반환

# 약한 감독 신호 생성 함수
# 최적화된 약한 감독 신호 생성 함수 (배치 처리 적용)
def generate_weak_supervision_pairs(job_fields, papers, template_sentences=None, simcse_model=None, tokenizer=None, device=None):
    """키워드, 카테고리, 텍스트 유사도 등을 활용한 약한 감독 신호 생성 - 배치 처리 최적화"""
    print("Generating weak supervision pairs with batch processing...")
    pairs = []

    # 템플릿 문장 기반 관계 추출
    template_relations = {}
    if template_sentences:
        print("Extracting relations from template sentences...")
        # 개선된 관계 추출 함수 사용
        template_sentences, template_relations, matching_stats = generate_templates_with_target_success_rate(
            job_fields,
            papers,
            templates,
            target_min=TEMPLATE_TARGET_MIN,  # 목표 최소 성공률
            target_max=TEMPLATE_TARGET_MAX,  # 목표 최대 성공률
            max_attempts=TEMPLATE_TARGET_TRY_LIMIT  # 최대 시도 횟수
        )
        # template_relations = extract_relations_from_templates(
        #     template_sentences,
        #     job_fields,
        #     papers,
        #     tokenizer=tokenizer,
        #     model=simcse_model,
        #     device=device
        # )

    print(f"Extracted {len(template_relations)} relations from template sentences")

    # 모델 사용 여부 확인 - Hi-BERT 사용
    use_semantic_similarity = simcse_model is not None and tokenizer is not None and device is not None

    if use_semantic_similarity:
        print("Using Hi-BERT for semantic similarity calculation")
        simcse_model.eval()  # 평가 모드 설정

    # MAX_JOB_COUNT 적용
    job_count = min(MAX_JOB_COUNT if MAX_JOB_COUNT is not None else 300, len(job_fields))
    job_sample = job_fields[:job_count]

    # 논문 샘플 수 감소 (최적화)
    max_papers = min(500, len(papers))  # 1000개에서 500개로 감소
    paper_sample = random.sample(papers, max_papers)

    print(f"Processing {len(job_sample)} jobs and {len(paper_sample)} papers")

    # 배치 단위 처리를 위한 준비
    batch_size = 32  # 배치 크기
    job_batches = [job_sample[i:i+batch_size] for i in range(0, len(job_sample), batch_size)]

    all_pairs = []

    for batch_idx, job_batch in enumerate(tqdm(job_batches, desc="Processing job batches")):
        # 배치 내 모든 채용공고의 키워드, 전공, 스킬 정보 준비
        batch_job_data = []
        batch_job_texts = []
        batch_job_input_ids = []
        batch_job_attention_masks = []

        for job in job_batch:
            job_keywords = set([k.lower() for k in job.get('keywords', []) if k])
            job_majors = set([m.lower() for m in job.get('majors', []) if m])
            job_skills = set([s.lower() for s in job.get('skills', []) if s])

            # 키워드가 없는 경우 필드명에서 추출
            if not job_keywords and job.get('field'):
                field_tokens = [w.lower() for w in job['field'].split() if len(w) > 2 and w.lower() not in STOPWORDS]
                job_keywords = set(field_tokens)

            # 기술 용어를 키워드에 추가
            for skill in job_skills:
                if len(skill) > 2 and skill.lower() not in STOPWORDS:
                    job_keywords.add(skill.lower())

            # 채용공고 텍스트 준비
            job_text = f"분야: {job.get('field', '')} 업무: {job.get('duties', '')} 자격요건: {job.get('qualifications', '')} " \
                    f"전공: {' '.join(job.get('majors', []))} 키워드: {' '.join(job.get('keywords', []))} 기술: {' '.join(job.get('skills', []))}"

            batch_job_data.append({
                'job': job,
                'keywords': job_keywords,
                'majors': job_majors,
                'skills': job_skills
            })
            batch_job_texts.append(job_text)

        # 채용공고 임베딩 배치 계산 (모델 사용 시)
        batch_job_embeddings = None
        if use_semantic_similarity:
            # 토큰화
            job_encodings = tokenizer(
                batch_job_texts,
                truncation=True,
                max_length=MAX_LENGTH,
                padding='max_length',
                return_tensors='pt'
            )

            batch_job_input_ids = job_encodings['input_ids'].to(device)
            batch_job_attention_masks = job_encodings['attention_mask'].to(device)

            # 배치 임베딩 계산
            with torch.no_grad():
                # 배치가 너무 크면 서브 배치로 나누어 처리
                sub_batch_size = 16  # GPU 메모리에 맞게 조정
                batch_job_embeddings = []

                for i in range(0, len(batch_job_input_ids), sub_batch_size):
                    sub_input_ids = batch_job_input_ids[i:i+sub_batch_size]
                    sub_attention_masks = batch_job_attention_masks[i:i+sub_batch_size]
                    sub_embeddings = simcse_model.encode_job(sub_input_ids, sub_attention_masks)
                    batch_job_embeddings.append(sub_embeddings)

                # 서브 배치 임베딩 합치기
                batch_job_embeddings = torch.cat(batch_job_embeddings, dim=0)

        # 논문 처리 (배치 단위)
        # 논문을 여러 배치로 나누어 처리
        paper_batch_size = 50  # 논문 배치 크기
        paper_batches = [paper_sample[i:i+paper_batch_size] for i in range(0, len(paper_sample), paper_batch_size)]

        for job_idx, job_data in enumerate(batch_job_data):
            job = job_data['job']
            job_keywords = job_data['keywords']
            job_majors = job_data['majors']
            job_skills = job_data['skills']

            # 현재 채용공고의 임베딩
            job_emb = None
            if batch_job_embeddings is not None:
                job_emb = batch_job_embeddings[job_idx].unsqueeze(0)  # [1, dim]

            # 점수화된 논문 저장
            scored_papers = []

            # 논문 배치 처리
            for paper_batch in paper_batches:
                batch_paper_data = []
                batch_paper_texts = []

                for paper in paper_batch:
                    paper_keywords = set([k.lower() for k in paper.get('keywords', []) if k])

                    # 논문 키워드가 없는 경우 제목에서 추출
                    if not paper_keywords and paper.get('title'):
                        title_tokens = [w.lower() for w in paper['title'].split() if len(w) > 2 and w.lower() not in STOPWORDS]
                        paper_keywords = set(title_tokens)

                    # 초록에서 중요 단어 추출
                    if paper.get('abstract'):
                        abstract_words = [w.lower() for w in paper.get('abstract', '').split()
                                        if len(w) > 3 and w.lower() not in STOPWORDS]
                        # 빈도수가 높은 단어 추가 (최대 3개)
                        word_count = {}
                        for word in abstract_words:
                            word_count[word] = word_count.get(word, 0) + 1

                        # 빈도수 기준 상위 3개 단어 추가
                        top_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:3]
                        for word, _ in top_words:
                            paper_keywords.add(word)

                    paper_text = f"제목: {paper.get('title', '')} 키워드: {', '.join(paper_keywords)} 초록: {paper.get('abstract', '')}"

                    batch_paper_data.append({
                        'paper': paper,
                        'keywords': paper_keywords
                    })
                    batch_paper_texts.append(paper_text)

                # 논문 임베딩 배치 계산 (모델 사용 시)
                if use_semantic_similarity and job_emb is not None:
                    # 토큰화
                    paper_encodings = tokenizer(
                        batch_paper_texts,
                        truncation=True,
                        max_length=MAX_LENGTH,
                        padding='max_length',
                        return_tensors='pt'
                    )

                    batch_paper_input_ids = paper_encodings['input_ids'].to(device)
                    batch_paper_attention_masks = paper_encodings['attention_mask'].to(device)

                    # 배치 임베딩 계산
                    with torch.no_grad():
                        # 배치가 너무 크면 서브 배치로 나누어 처리
                        sub_batch_size = 16  # GPU 메모리에 맞게 조정
                        batch_paper_embeddings = []

                        for i in range(0, len(batch_paper_input_ids), sub_batch_size):
                            sub_input_ids = batch_paper_input_ids[i:i+sub_batch_size]
                            sub_attention_masks = batch_paper_attention_masks[i:i+sub_batch_size]
                            sub_embeddings = simcse_model.encode_paper(sub_input_ids, sub_attention_masks)
                            batch_paper_embeddings.append(sub_embeddings)

                        # 서브 배치 임베딩 합치기
                        batch_paper_embeddings = torch.cat(batch_paper_embeddings, dim=0)

                        # 일괄 유사도 계산 (batch matrix multiplication)
                        # job_emb: [1, dim], batch_paper_embeddings: [batch_size, dim]
                        # similarity: [1, batch_size]
                        similarities = torch.matmul(job_emb, batch_paper_embeddings.t())
                        similarities = (similarities + 1) / 2  # 범위 조정 (-1~1 -> 0~1)

                # 배치 내 각 논문 처리
                for i, paper_data in enumerate(batch_paper_data):
                    paper = paper_data['paper']
                    paper_keywords = paper_data['keywords']

                    # 키워드 매칭 점수
                    keyword_overlap = len(job_keywords.intersection(paper_keywords))
                    keyword_match = min(keyword_overlap * 0.2, 0.4)  # 최대 0.4점

                    # 키워드 부분 매칭 점수 (최적화)
                    partial_match = 0
                    for j_kw in job_keywords:
                        if len(j_kw) <= 2:
                            continue
                        for p_kw in paper_keywords:
                            if len(p_kw) <= 2:
                                continue
                            if j_kw in p_kw or p_kw in j_kw:
                                partial_match += 0.1
                                break
                    partial_match = min(partial_match, 0.3)  # 최대 0.3점

                    # 전공-카테고리 매칭 점수
                    major_match = 0
                    if paper.get('category'):
                        paper_category = paper['category'].lower()
                        for major in job_majors:
                            if major in paper_category:
                                major_match = 0.15
                                break

                    # 기술-초록 매칭 점수
                    skill_match = 0
                    if paper.get('abstract'):
                        abstract_lower = paper['abstract'].lower()
                        for skill in job_skills:
                            if skill in abstract_lower:
                                skill_match += 0.15
                                break

                    # 분야-제목 매칭 점수
                    field_match = 0
                    if job.get('field') and paper.get('title'):
                        field_words = [w.lower() for w in job['field'].split() if w.lower() not in STOPWORDS and len(w) > 2]
                        title_lower = paper['title'].lower()
                        for word in field_words:
                            if word in title_lower:
                                field_match += 0.15
                                break

                    # 템플릿 기반 관계 점수
                    template_match = 0
                    if template_relations:
                        key = (job['job_id'], paper['paper_id'])
                        if key in template_relations:
                            template_match = min(template_relations[key] * 0.2, 0.2)  # 최대 0.2점

                    # 의미적 유사도 점수
                    semantic_similarity = 0
                    if use_semantic_similarity and similarities is not None:
                        semantic_similarity = similarities[0, i].item()

                    # 종합 점수 계산
                    if use_semantic_similarity:
                        score = (
                            (keyword_match * 0.2) +
                            (partial_match * 0.2) +
                            (major_match * 0.1) +
                            (skill_match * 0.15) +
                            (field_match * 0.15) +
                            (template_match * 0.1) +
                            (semantic_similarity * 0.3)
                        )
                    else:
                        score = (
                            (keyword_match * 0.25) +
                            (partial_match * 0.25) +
                            (major_match * 0.15) +
                            (skill_match * 0.15) +
                            (field_match * 0.15) +
                            (template_match * 0.15)
                        )

                    # 최소 점수 부여 (일부 랜덤 매칭)
                    if score == 0 and random.random() < 0.1:
                        score = 0.1

                    # 최소 점수 이상인 경우만 후보로 추가
                    if score > 0:
                        scored_papers.append((paper, score))

            # 점수로 정렬
            scored_papers.sort(key=lambda x: x[1], reverse=True)

            # 각 직무당 페어 생성 (최소 1개는 생성)
            if scored_papers:
                # 상위 5개를 긍정 샘플로 간주
                for i, (paper, score) in enumerate(scored_papers[:5]):
                    all_pairs.append({
                        'job': job,
                        'paper': paper,
                        'score': score,
                        'is_positive': 1
                    })

                # 그 다음 3개 논문 (하드 네거티브)
                for i, (paper, score) in enumerate(scored_papers[5:8]):
                    all_pairs.append({
                        'job': job,
                        'paper': paper,
                        'score': 0.0,  # 부정 샘플은 점수를 0으로 설정
                        'is_positive': 0
                    })
            else:
                # 매칭된 논문이 없는 경우, 랜덤 논문 선택
                random_paper = random.choice(paper_sample)
                all_pairs.append({
                    'job': job,
                    'paper': random_paper,
                    'score': 0.15,
                    'is_positive': 1
                })

    # 최대 페어 수 제한
    max_pairs = 15000
    if len(all_pairs) > max_pairs:
        all_pairs = random.sample(all_pairs, max_pairs)

    print(f"Generated {len(all_pairs)} weak supervision pairs")
    return all_pairs

# Hi-BERT 이중 인코더 모델 정의
class DualEncoderModel(torch.nn.Module):
    def __init__(self, base_model, projection_dim=768):
        super(DualEncoderModel, self).__init__()
        # 모델 이름 속성 추가 - Hi-BERT 식별용
        self.model_name = "Hi-BERT"

        # BERT 모델
        self.bert = base_model

        # 채용정보와 논문을 위한 별도의 프로젝션 레이어 (더 깊은 네트워크로 변경)
        self.job_projection = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.ReLU(),
            # torch.nn.Dropout(0.2),  # 드롭아웃 추가
            torch.nn.Linear(768, projection_dim)
        )

        self.paper_projection = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.ReLU(),
            torch.nn.Linear(768, projection_dim)
        )

        # LayerNorm 추가 (정규화 성능 향상)
        self.job_norm = torch.nn.LayerNorm(projection_dim)
        self.paper_norm = torch.nn.LayerNorm(projection_dim)

        # 드롭아웃 레이어
        self.dropout = torch.nn.Dropout(HBN_DROPOUT)

        # 가중치 초기화 추가
        self._init_weights()

    def _init_weights(self):
        """가중치 초기화 함수"""
        for module in [self.job_projection, self.paper_projection]:
            for m in module.modules():
                if isinstance(m, torch.nn.Linear):
                    torch.nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        torch.nn.init.constant_(m.bias, 0)

    def encode_job(self, input_ids, attention_mask):
        # 채용정보 인코딩
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        if hasattr(outputs, 'pooler_output'):
            pooled_output = outputs.pooler_output
        else:
            pooled_output = outputs[1]  # 튜플로 반환되는 경우

        pooled_output = self.dropout(pooled_output)
        job_emb = self.job_projection(pooled_output)
        job_emb = self.job_norm(job_emb)  # LayerNorm 적용

        # L2 정규화
        job_emb = job_emb / (torch.norm(job_emb, dim=1, keepdim=True) + 1e-12)
        return job_emb

    def encode_paper(self, input_ids, attention_mask):
        # 논문 인코딩
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        if hasattr(outputs, 'pooler_output'):
            pooled_output = outputs.pooler_output
        else:
            pooled_output = outputs[1]  # 튜플로 반환되는 경우

        pooled_output = self.dropout(pooled_output)
        paper_emb = self.paper_projection(pooled_output)
        paper_emb = self.paper_norm(paper_emb)  # LayerNorm 적용

        # L2 정규화
        paper_emb = paper_emb / (torch.norm(paper_emb, dim=1, keepdim=True) + 1e-12)
        return paper_emb

    def forward(self, job_input_ids, job_attention_mask, paper_input_ids, paper_attention_mask):
        # 채용정보와 논문 모두 인코딩
        job_emb = self.encode_job(job_input_ids, job_attention_mask)
        paper_emb = self.encode_paper(paper_input_ids, paper_attention_mask)

        return job_emb, paper_emb

# 대조 손실 함수 (개선된 버전)
def contrastive_loss(job_emb, paper_emb, temperature=TEMPERATURE, margin=MARGIN):
    """인배치 대조 손실 함수 - 마진 손실 추가"""
    # 벡터 정규화 (이미 모델에서 정규화했으므로 중복일 수 있음)
    job_emb = job_emb / job_emb.norm(dim=1, keepdim=True)
    paper_emb = paper_emb / paper_emb.norm(dim=1, keepdim=True)

    # 코사인 유사도 행렬 계산 (batch_size x batch_size)
    logits = torch.matmul(job_emb, paper_emb.t()) / temperature

    # 대각선 요소가 양성 쌍이 되도록 레이블 설정
    labels = torch.arange(logits.size(0), device=logits.device)

    # 교차 엔트로피 손실 계산
    ce_loss = torch.nn.CrossEntropyLoss()(logits, labels)

    # 마진 손실 추가 (하드 네거티브 샘플링)
    batch_size = job_emb.size(0)
    mask = torch.eye(batch_size, device=job_emb.device)

    # 양성 쌍의 유사도
    positive_pairs = torch.sum(job_emb * paper_emb, dim=1)

    # 음성 쌍의 최대 유사도 (하드 네거티브)
    negative_pairs = torch.matmul(job_emb, paper_emb.t()) * (1 - mask)
    hardest_negatives, _ = torch.max(negative_pairs, dim=1)

    # 마진 손실
    margin_loss = torch.nn.functional.relu(margin - positive_pairs + hardest_negatives).mean()

    # 최종 손실 (CE 손실 + 마진 손실)
    total_loss = ce_loss + margin_loss

    return total_loss

# MLM 학습 함수
def train_domain_adapted_mlm(job_fields, papers, tokenizer, model, output_dir, template_sentences=None, reset_checkpoints=False):
    """도메인 적응을 위한 MLM 학습 (체크포인트 기능 추가)"""
    print("\n=== 도메인 적응 MLM 학습 시작 ===")

    # 체크포인트 디렉토리 확인
    mlm_checkpoint_dir = f"{output_dir}/mlm_checkpoints"
    os.makedirs(mlm_checkpoint_dir, exist_ok=True)

    # 체크포인트 정보 파일
    checkpoint_info_path = f"{mlm_checkpoint_dir}/checkpoint_info.json"

    # 체크포인트 초기화가 요청된 경우 기존 체크포인트 정보 파일 삭제
    if reset_checkpoints and os.path.exists(checkpoint_info_path):
        os.remove(checkpoint_info_path)
        print("MLM 체크포인트 정보 파일이 초기화되었습니다.")

    # 체크포인트 확인
    checkpoint_exists = os.path.exists(checkpoint_info_path) and not reset_checkpoints
    resume_training = False
    start_epoch = 0
    global_step = 0

    # 특수 토큰 추가
    special_tokens = {"additional_special_tokens": ["[JOB]", "[PAPER]", "[TEMPLATE]"]}
    tokenizer.add_special_tokens(special_tokens)
    model.resize_token_embeddings(len(tokenizer))

    # 텍스트 구성
    mlm_texts = []

    # 채용공고 텍스트 추가 (데이터 양 제한)
    job_sample = random.sample(job_fields, min(500, len(job_fields)))
    for job in job_sample:
        job_text = f"[JOB] {job['job_text']}"
        mlm_texts.append(job_text)

    # 논문 텍스트 추가 (데이터 양 제한)
    paper_sample = random.sample(papers, min(2000, len(papers)))
    for paper in paper_sample:
        paper_text = f"[PAPER] {paper['paper_text']}"
        mlm_texts.append(paper_text)

    # 템플릿 문장 추가 (데이터 양 제한)
    if template_sentences:
        template_sample = random.sample(template_sentences, min(MAX_TEMPLATE_COUNT, len(template_sentences)))
        print(f"Adding {len(template_sample)} template sentences to MLM training data")
        for sentence in template_sample:
            template_text = f"[TEMPLATE] {sentence}"
            mlm_texts.append(template_text)

    random.shuffle(mlm_texts)

    # 데이터셋 및 데이터로더 구성
    train_texts, val_texts = train_test_split(mlm_texts, test_size=0.1, random_state=42)

    # MLM 데이터 콜레이터
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )

    # 학습 준비
    model.to(DEVICE)
    optimizer = AdamW(model.parameters(), lr=MLM_LEARNING_RATE)

    # 총 학습 스텝 계산
    train_dataset = DomainTextDataset(train_texts, tokenizer, MAX_LENGTH)
    val_dataset = DomainTextDataset(val_texts, tokenizer, MAX_LENGTH)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    total_steps = len(train_dataloader) * MLM_EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=total_steps // 10,
        num_training_steps=total_steps
    )

    # 체크포인트 로드 시도
    if checkpoint_exists:
        checkpoint, loaded_step = load_latest_checkpoint(
            mlm_checkpoint_dir, model, optimizer, scheduler)
        if checkpoint:
            # 매우 큰 loaded_step 값(999999)을 받으면 MLM 학습 완료로 처리
            if loaded_step >= 999999:
                print("MLM training already completed. Skipping to next stage.")
                return model, tokenizer

            resume_training = True
            # 에포크 계산 (몇 번째 에포크부터 시작할지)
            # 숫자인지 확인 후 계산
            if isinstance(loaded_step, int) and loaded_step > 0:
                start_epoch = loaded_step // len(train_dataloader)
                global_step = loaded_step
                print(f"Resuming MLM training from epoch {start_epoch+1}")
            else:
                # 적절한 스텝 값이 아닌 경우 처음부터 시작
                start_epoch = 0
                global_step = 0
                print("Invalid step value. Starting MLM training from scratch.")

    scaler = GradScaler()
    best_val_loss = float('inf')

    # 훈련 루프
    for epoch in range(start_epoch, MLM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{MLM_EPOCHS}")
        model.train()
        total_train_loss = 0
        train_progress_bar = tqdm(train_dataloader, desc=f"Training")

        # 학습
        for batch_idx, batch in enumerate(train_progress_bar):
            # 체크포인트에서 재개하는 경우 이미 처리한 배치는 건너뛰기
            if resume_training and epoch == start_epoch and batch_idx < (global_step % len(train_dataloader)):
                continue

            # 데이터를 디바이스로 이동
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)

            # MLM을 위한 라벨 생성
            batch_inputs = {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': input_ids.clone()
            }

            # 마스킹된 토큰 생성 (DataCollator로 자동화)
            masked_inputs = data_collator(
                [{'input_ids': ids, 'attention_mask': mask}
                 for ids, mask in zip(input_ids, attention_mask)]
            )

            # 모든 데이터를 디바이스로 이동
            for k, v in masked_inputs.items():
                masked_inputs[k] = v.to(DEVICE)

            # 자동 혼합 정밀도로 순전파 및 손실 계산
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(**masked_inputs)
                loss = outputs.loss / ACCUMULATION_STEPS

            # 스케일링된 역전파
            scaler.scale(loss).backward()

            # 원래 손실 (스케일링 전) 추적
            total_train_loss += loss.item() * ACCUMULATION_STEPS

            # 그라디언트 누적 후 업데이트
            if (batch_idx + 1) % ACCUMULATION_STEPS == 0 or (batch_idx + 1) == len(train_dataloader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()

                global_step += 1

                # 100 스텝마다 체크포인트 저장
                if global_step % 100 == 0:
                    save_checkpoint(
                        mlm_checkpoint_dir,
                        global_step,
                        model,
                        optimizer,
                        scheduler,
                        tokenizer,
                        loss.item() * ACCUMULATION_STEPS
                    )

            train_progress_bar.set_postfix({'loss': loss.item() * ACCUMULATION_STEPS, 'step': global_step})

            # 체크포인트에서 재개한 경우, 첫 배치 후 resume_training 해제
            if batch_idx == 0 and epoch == start_epoch and resume_training:
                resume_training = False

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # 검증
        model.eval()
        total_val_loss = 0
        val_progress_bar = tqdm(val_dataloader, desc=f"Validation")

        with torch.no_grad():
            for batch in val_progress_bar:
                input_ids = batch['input_ids'].to(DEVICE)
                attention_mask = batch['attention_mask'].to(DEVICE)

                masked_inputs = data_collator(
                    [{'input_ids': ids, 'attention_mask': mask}
                     for ids, mask in zip(input_ids, attention_mask)]
                )

                for k, v in masked_inputs.items():
                    masked_inputs[k] = v.to(DEVICE)

                outputs = model(**masked_inputs)
                loss = outputs.loss

                total_val_loss += loss.item()
                val_progress_bar.set_postfix({'loss': loss.item()})

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Validation loss: {avg_val_loss:.4f}")

        # 최고 모델 저장
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            print(f"Saving best model to {output_dir}/mlm_best_model")

            # 모델 및 토크나이저 저장
            model.save_pretrained(f"{output_dir}/mlm_best_model")
            tokenizer.save_pretrained(f"{output_dir}/mlm_best_model")

            # 에포크 체크포인트 저장
            save_checkpoint(
                mlm_checkpoint_dir,
                f"epoch_{epoch+1}",
                model,
                optimizer,
                scheduler,
                tokenizer,
                avg_val_loss
            )

    print("=== 도메인 적응 MLM 학습 완료 ===")
    return model, tokenizer


# 대조학습 모델 훈련 함수
# 최적화된 대조학습 함수
def train_contrastive_learning(job_fields, papers, base_model, tokenizer, output_dir, template_sentences=None, reset_checkpoints=False):
    """대조학습 기반의 Hi-BERT 모델 훈련 (배치 처리 및 병렬 처리 최적화)"""
    print("\n=== 대조학습 기반 Hi-BERT 훈련 시작 ===")

    # 체크포인트 디렉토리 확인
    cl_checkpoint_dir = f"{output_dir}/cl_checkpoints"
    os.makedirs(cl_checkpoint_dir, exist_ok=True)

    # 체크포인트 정보 파일
    checkpoint_info_path = f"{cl_checkpoint_dir}/checkpoint_info.json"

    # 체크포인트 초기화가 요청된 경우 기존 체크포인트 정보 파일 삭제
    if reset_checkpoints and os.path.exists(checkpoint_info_path):
        os.remove(checkpoint_info_path)
        print("대조학습 체크포인트 정보 파일이 초기화되었습니다.")

    # 체크포인트 확인
    checkpoint_exists = os.path.exists(checkpoint_info_path) and not reset_checkpoints
    resume_training = False
    start_epoch = 0
    global_step = 0

    # 약한 감독 신호 생성 (데이터 양 제한)
    # 채용공고 및 논문 샘플링 - 최적화된 샘플 수
    max_jobs_for_cl = min(300, len(job_fields))  # 500에서 300으로 감소
    max_papers_for_cl = min(1000, len(papers))   # 2000에서 1000으로 감소
    max_templates_for_cl = int(min(MAX_TEMPLATE_COUNT, len(template_sentences) if template_sentences else 0) * 0.7)

    job_sample = random.sample(job_fields, min(max_jobs_for_cl, len(job_fields)))
    paper_sample = random.sample(papers, min(max_papers_for_cl, len(papers)))

    print(f"Generating weak supervision pairs with {len(job_sample)} jobs and {len(paper_sample)} papers...")
    # 템플릿 문장도 제한된 수만 사용
    template_sample = None
    if template_sentences:
        template_sample = random.sample(template_sentences, min(max_templates_for_cl, len(template_sentences)))

    # 약한 감독 신호 생성 함수 호출
    pairs_file = f"{output_dir}/weak_supervision_pairs.pkl"

    # 이미 생성된 페어 파일이 있고 초기화하지 않는 경우 로드
    if os.path.exists(pairs_file) and not reset_checkpoints:
        print(f"Loading pre-generated pairs from {pairs_file}")
        with open(pairs_file, 'rb') as f:
            pairs = pickle.load(f)
    else:
        # 이중 인코더 모델 초기화 - 약한 감독 신호 생성에 사용
        temp_model = DualEncoderModel(base_model, projection_dim=PROJECTION_DIM)
        temp_model.to(DEVICE)

        # 최적화된 페어 생성 함수 호출
        pairs = generate_weak_supervision_pairs(
            job_sample,  # 샘플링된 채용공고 사용
            paper_sample,  # 샘플링된 논문 사용
            template_sample,  # 샘플링된 템플릿 사용
            simcse_model=temp_model,
            tokenizer=tokenizer,
            device=DEVICE
        )

        # 메모리 정리
        del temp_model
        torch.cuda.empty_cache()

        # 페어 저장
        with open(pairs_file, 'wb') as f:
            pickle.dump(pairs, f)

    # 검증 세트 분리
    train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=42)

    # 데이터 로더 설정 최적화
    # 병렬 데이터 로딩 및 GPU 메모리 전송 최적화
    train_dataset = ContrastivePairDataset(train_pairs, tokenizer, MAX_LENGTH)
    val_dataset = ContrastivePairDataset(val_pairs, tokenizer, MAX_LENGTH)

    hibert_batch_size = 16  # 배치 크기 유지

    # 데이터 로더 최적화
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=hibert_batch_size,
        shuffle=True,
        num_workers=4,        # 병렬 데이터 로딩 워커 수 증가
        pin_memory=True,      # GPU 메모리 전송 최적화
        prefetch_factor=2,    # 미리 배치 준비
        persistent_workers=True # 워커 유지 (성능 향상)
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=hibert_batch_size,
        num_workers=4,
        pin_memory=True,
        prefetch_factor=2,
        persistent_workers=True
    )

    # Hi-BERT 이중 인코더 모델 초기화
    model = DualEncoderModel(base_model, projection_dim=PROJECTION_DIM)

    # 모델을 디바이스로 이동
    model.to(DEVICE)

    # 옵티마이저 및 스케줄러 설정
    optimizer = AdamW(
        model.parameters(),
        lr=CL_LEARNING_RATE,
        weight_decay=0.01
    )

    total_steps = len(train_dataloader) * CL_EPOCHS

    # 코사인 스케줄러 유지
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=total_steps,
        eta_min=1e-6
    )

    # 체크포인트 로드 시도
    if checkpoint_exists:
        checkpoint, loaded_step = load_latest_checkpoint(
            cl_checkpoint_dir, model, optimizer, scheduler)
        if checkpoint:
            resume_training = True
            # 에포크 계산
            start_epoch = loaded_step // len(train_dataloader)
            global_step = loaded_step
            print(f"Resuming contrastive learning from epoch {start_epoch+1}")

    scaler = GradScaler()
    best_val_loss = float('inf')

    # 학습률 기록용 리스트
    learning_rates = []

    # 훈련 루프
    for epoch in range(start_epoch, CL_EPOCHS):
        print(f"\nEpoch {epoch+1}/{CL_EPOCHS}")
        model.train()
        total_train_loss = 0
        train_progress_bar = tqdm(train_dataloader, desc=f"Training")

        # 배치 처리 최적화
        for batch_idx, batch in enumerate(train_progress_bar):
            # 체크포인트에서 재개하는 경우 이미 처리한 배치는 건너뛰기
            if resume_training and epoch == start_epoch and batch_idx < (global_step % len(train_dataloader)):
                continue

            # 데이터를 디바이스로 이동 (비동기 전송 최적화)
            job_input_ids = batch['job_input_ids'].to(DEVICE, non_blocking=True)
            job_attention_mask = batch['job_attention_mask'].to(DEVICE, non_blocking=True)
            paper_input_ids = batch['paper_input_ids'].to(DEVICE, non_blocking=True)
            paper_attention_mask = batch['paper_attention_mask'].to(DEVICE, non_blocking=True)

            # 자동 혼합 정밀도로 순전파 및 손실 계산
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                job_emb, paper_emb = model(
                    job_input_ids, job_attention_mask,
                    paper_input_ids, paper_attention_mask
                )

                # 개선된 대조 손실 함수 사용
                loss = contrastive_loss(job_emb, paper_emb, temperature=TEMPERATURE, margin=MARGIN) / ACCUMULATION_STEPS

            # NaN 체크 및 처리
            if torch.isnan(loss):
                print(f"Warning: NaN loss detected at batch {batch_idx}, skipping batch")
                optimizer.zero_grad()
                continue

            # 스케일링된 역전파
            scaler.scale(loss).backward()

            # 원래 손실 (스케일링 전) 추적
            total_train_loss += loss.item() * ACCUMULATION_STEPS

            # 그라디언트 누적 후 업데이트
            if (batch_idx + 1) % ACCUMULATION_STEPS == 0 or (batch_idx + 1) == len(train_dataloader):
                # 그라디언트 클리핑
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # 옵티마이저 스텝
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()

                # 현재 학습률 기록
                learning_rates.append(optimizer.param_groups[0]['lr'])

                optimizer.zero_grad()

                global_step += 1

                # 50 스텝마다 체크포인트 저장 (30에서 50으로 변경)
                if global_step % 50 == 0:
                    save_checkpoint(
                        cl_checkpoint_dir,
                        global_step,
                        model,
                        optimizer,
                        scheduler,
                        None,
                        loss.item() * ACCUMULATION_STEPS
                    )

            train_progress_bar.set_postfix({
                'loss': loss.item() * ACCUMULATION_STEPS,
                'step': global_step,
                'lr': optimizer.param_groups[0]['lr']
            })

            # 체크포인트에서 재개한 경우, 첫 배치 후 resume_training 해제
            if batch_idx == 0 and epoch == start_epoch and resume_training:
                resume_training = False

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # 검증
        model.eval()
        total_val_loss = 0
        val_progress_bar = tqdm(val_dataloader, desc=f"Validation")

        with torch.no_grad():
            for batch in val_progress_bar:
                job_input_ids = batch['job_input_ids'].to(DEVICE, non_blocking=True)
                job_attention_mask = batch['job_attention_mask'].to(DEVICE, non_blocking=True)
                paper_input_ids = batch['paper_input_ids'].to(DEVICE, non_blocking=True)
                paper_attention_mask = batch['paper_attention_mask'].to(DEVICE, non_blocking=True)

                job_emb, paper_emb = model(
                    job_input_ids, job_attention_mask,
                    paper_input_ids, paper_attention_mask
                )

                # 개선된 대조 손실 함수 사용
                loss = contrastive_loss(job_emb, paper_emb, temperature=TEMPERATURE, margin=MARGIN)

                # NaN 체크
                if not torch.isnan(loss):
                    total_val_loss += loss.item()
                    val_progress_bar.set_postfix({'loss': loss.item()})

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Validation loss: {avg_val_loss:.4f}")

        # 최고 모델 저장
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            print(f"Saving best model to {output_dir}/cl_best_model")

            # 에포크 체크포인트 저장
            save_checkpoint(
                cl_checkpoint_dir,
                f"epoch_{epoch+1}_best",
                model,
                optimizer,
                scheduler,
                None,
                avg_val_loss
            )

            # 최고 모델 저장
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss,
                'epoch': epoch
            }, f"{output_dir}/cl_best_model.pt")

            # Hi-BERT를 위한 추가 저장 (모델 가중치만 따로 저장)
            torch.save(model.state_dict(), f"{output_dir}/hibert_weights.pt")

    print("=== Hi-BERT 대조학습 훈련 완료 ===")
    return model

# 모델 검증 함수
# 최적화된 모델 검증 함수 (배치 처리 적용)
def validate_model(model, tokenizer, job_fields, papers, device):
    """학습된 Hi-BERT 모델의 성능 검증 (배치 처리 최적화)"""
    print("\n=== Hi-BERT 모델 검증 시작 (배치 처리 최적화) ===")

    # 검증에 사용할 샘플 수 제한
    test_job_fields = []

    # 유효한 채용공고만 필터링
    for job in job_fields[:50]:  # 상위 50개에서 검사
        if job.get('field'):  # 필드가 있는 경우만 포함
            test_job_fields.append(job)
            if len(test_job_fields) >= 20:  # 테스트에 사용할 20개만 선택
                break

    # 유효한 채용공고가 없는 경우
    if not test_job_fields:
        print("유효한 채용공고가 없습니다. 검증을 건너뜁니다.")
        return {'mrr': 0, 'ndcg': 0, 'precision@5': 0, 'precision@10': 0}

    # 최대 100개 논문
    test_papers = papers[:100]

    # 모델이 평가 모드인지 확인
    model.eval()

    # 모델이 올바른 장치에 있는지 확인
    if next(model.parameters()).device != device:
        model = model.to(device)

    # 모델 이름 가져오기 (Hi-BERT 여부 확인)
    model_name = getattr(model, 'model_name', "")
    is_hibert = model_name == "Hi-BERT"  # 모델 이름이 "Hi-BERT"인지 정확히 확인

    # 평가 지표 초기화
    mrr_scores = []
    ndcg_scores = []
    precision_at_5 = []
    precision_at_10 = []

    # 채용공고 배치 처리
    job_batch_size = 4  # 배치당 채용공고 수
    job_batches = [test_job_fields[i:i+job_batch_size] for i in range(0, len(test_job_fields), job_batch_size)]

    # 논문 데이터를 미리 토큰화하여 재사용하기 위한 준비
    paper_batch_size = 20  # 논문 배치 크기
    paper_batches = [test_papers[i:i+paper_batch_size] for i in range(0, len(test_papers), paper_batch_size)]

    # 논문 인코딩 및 임베딩 캐시
    paper_encodings_cache = {}
    paper_embeddings_cache = {}

    print("논문 임베딩 사전 계산 중...")
    with torch.no_grad():
        for batch_idx, paper_batch in enumerate(paper_batches):
            # 논문 텍스트 준비
            paper_texts = [
                f"제목: {paper['title']} 키워드: {', '.join(paper.get('keywords', []))} 초록: {paper.get('abstract', '')}"
                for paper in paper_batch
            ]

            # 배치 토큰화
            paper_encodings = tokenizer(
                paper_texts,
                truncation=True,
                max_length=MAX_LENGTH,
                padding='max_length',
                return_tensors='pt'
            )

            # 디바이스로 이동
            paper_input_ids = paper_encodings['input_ids'].to(device)
            paper_attention_mask = paper_encodings['attention_mask'].to(device)

            # 논문 임베딩 계산
            paper_embeddings = model.encode_paper(paper_input_ids, paper_attention_mask)

            # 캐시에 저장
            for i, paper in enumerate(paper_batch):
                paper_id = paper['paper_id']
                paper_encodings_cache[paper_id] = {
                    'input_ids': paper_input_ids[i].unsqueeze(0),
                    'attention_mask': paper_attention_mask[i].unsqueeze(0)
                }
                paper_embeddings_cache[paper_id] = paper_embeddings[i].unsqueeze(0)

    print(f"논문 임베딩 {len(paper_embeddings_cache)}개 사전 계산 완료")

    # 채용공고 배치 처리
    for batch_idx, job_batch in enumerate(tqdm(job_batches, desc="채용공고 배치 평가")):
        # 채용공고 텍스트 준비
        job_texts = []
        for job in job_batch:
            job_text = f"분야: {job['field']} 업무: {job.get('duties', '')} 자격요건: {job.get('qualifications', '')} " \
                     f"전공: {' '.join(job.get('majors', []))} 키워드: {' '.join(job.get('keywords', []))} " \
                     f"기술: {' '.join(job.get('skills', []))}"
            job_texts.append(job_text)

        # 배치 토큰화
        job_encodings = tokenizer(
            job_texts,
            truncation=True,
            max_length=MAX_LENGTH,
            padding='max_length',
            return_tensors='pt'
        )

        # 디바이스로 이동
        job_input_ids = job_encodings['input_ids'].to(device)
        job_attention_mask = job_encodings['attention_mask'].to(device)

        # 채용공고 임베딩 계산
        with torch.no_grad():
            job_embeddings = model.encode_job(job_input_ids, job_attention_mask)

        # 배치의 각 채용공고 처리
        for job_idx, job in enumerate(job_batch):
            # 현재 채용공고 임베딩
            job_emb = job_embeddings[job_idx].unsqueeze(0)  # [1, dim]

            # 모든 논문과의 유사도 계산 (배치 처리)
            paper_scores = []

            # 논문 배치 처리
            for paper_batch in paper_batches:
                # 배치 내 논문 임베딩 수집
                batch_paper_embeddings = torch.cat([
                    paper_embeddings_cache[paper['paper_id']]
                    for paper in paper_batch
                ], dim=0)  # [batch_size, dim]

                # 배치 유사도 계산
                with torch.no_grad():
                    # [1, dim] x [batch_size, dim]T = [1, batch_size]
                    similarities = torch.matmul(job_emb, batch_paper_embeddings.transpose(0, 1))

                # 배치의 각 논문에 대한 유사도 및 관련성 계산
                for paper_idx, paper in enumerate(paper_batch):
                    similarity = similarities[0, paper_idx].item()

                    # Hi-BERT 모델일 경우 유사도 스케일링 적용
                    if is_hibert:
                        scaled_similarity = min(similarity * 5.0, 1.0)
                    else:
                        scaled_similarity = similarity

                    # 관련성 판단 - 유사도 임계값 조정
                    is_relevant = 0  # 기본값 설정

                    # Hi-BERT 모델일 경우 더 낮은 임계값 적용
                    similarity_threshold = 0.25 if is_hibert else 0.3

                    # 유사도 임계값 적용
                    if scaled_similarity > similarity_threshold:
                        is_relevant = 1

                    # 추가 관련성 판단 - 키워드 매칭
                    job_keywords = set([k.lower() for k in job.get('keywords', []) if k])
                    paper_keywords = set([k.lower() for k in paper.get('keywords', []) if k])

                    # 키워드가 없는 경우 필드나 제목에서 추출
                    if not job_keywords and job.get('field'):
                        job_keywords = set([w.lower() for w in job['field'].split()
                                         if len(w) > 2 and w.lower() not in STOPWORDS])

                    if not paper_keywords and paper.get('title'):
                        paper_keywords = set([w.lower() for w in paper['title'].split()
                                           if len(w) > 2 and w.lower() not in STOPWORDS])

                    # 키워드 정확 매칭
                    keyword_matches = job_keywords.intersection(paper_keywords)
                    exact_match = 1 if keyword_matches else 0

                    # 정확 매칭된 키워드 목록
                    matched_keywords = list(keyword_matches)

                    # 키워드 부분 매칭 (최적화 버전)
                    partial_matches = []
                    partial_match = 0
                    if job_keywords and paper_keywords:
                        for j_kw in job_keywords:
                            if partial_match:
                                break
                            if len(j_kw) <= 2:
                                continue
                            for p_kw in paper_keywords:
                                if len(p_kw) <= 2:
                                    continue
                                if (j_kw in p_kw or p_kw in j_kw) and j_kw != p_kw:
                                    partial_matches.append((j_kw, p_kw))
                                    partial_match = 1
                                    break

                    # 분야-제목 매칭 (Hi-BERT용 추가 검사)
                    field_title_match = 0
                    if job.get('field') and paper.get('title'):
                        field_words = [w.lower() for w in job['field'].split()
                                     if len(w) > 2 and w.lower() not in STOPWORDS]
                        paper_title = paper['title'].lower()
                        for word in field_words:
                            if word in paper_title:
                                field_title_match = 1
                                break

                    # 기술-초록 매칭 (Hi-BERT용 추가 검사)
                    skill_abstract_match = 0
                    matched_skill = ""
                    if job.get('skills') and paper.get('abstract'):
                        for skill in job.get('skills', []):
                            if skill.lower() in paper['abstract'].lower():
                                skill_abstract_match = 1
                                matched_skill = skill
                                break

                    # 종합 관련성 점수 계산
                    relevance_score = 0

                    # Hi-BERT 모델일 경우 의미적 관계 가중치 증가
                    if is_hibert:
                        relevance_score = (
                            (scaled_similarity * 0.3) +
                            (exact_match * 0.3) +
                            (partial_match * 0.2) +
                            (field_title_match * 0.2) +
                            (skill_abstract_match * 0.2)
                        )
                    else:
                        relevance_score = (
                            (scaled_similarity * 0.5) +
                            (exact_match * 0.3) +
                            (partial_match * 0.2)
                        )

                    # 관련성 종합 판단 - 점수 임계값 적용
                    relevance_threshold = 0.35 if is_hibert else 0.4
                    if relevance_score > relevance_threshold:
                        is_relevant = 1

                    # 관련성 정보 구성
                    relevance_info = "관련" if is_relevant == 1 else "비관련"
                    relevance_details = []

                    if scaled_similarity > 0.2:
                        relevance_details.append(f"유사도 {similarity:.4f}")

                    if exact_match:
                        relevance_details.append(f"키워드 일치 {len(matched_keywords)}개 ({', '.join(matched_keywords[:3])})")

                    if partial_match:
                        relevance_details.append(f"부분 일치 {len(partial_matches)}개")

                    if is_hibert:
                        if field_title_match:
                            relevance_details.append("분야-제목 매칭")

                        if skill_abstract_match:
                            relevance_details.append(f"기술-초록 매칭 ({matched_skill})")

                    relevance_str = f"{relevance_info} ({', '.join(relevance_details)})" if relevance_details else relevance_info

                    paper_scores.append({
                        'paper': paper,
                        'similarity': similarity,
                        'scaled_similarity': scaled_similarity,
                        'is_relevant': is_relevant,
                        'relevance_score': relevance_score,
                        'relevance_str': relevance_str,
                        'keyword_match': exact_match,
                        'partial_match': partial_match,
                        'field_title_match': field_title_match,
                        'skill_abstract_match': skill_abstract_match
                    })

            # 관련성 점수에 따라 정렬
            paper_scores.sort(key=lambda x: x['relevance_score'], reverse=True)

            # MRR 계산
            mrr = 0
            for i, ps in enumerate(paper_scores, 1):
                if ps['is_relevant'] == 1:
                    mrr = 1.0 / i
                    break

            # NDCG@10 계산
            relevance_scores = [ps['is_relevant'] for ps in paper_scores[:10]]
            if relevance_scores and any(relevance_scores):
                ideal_scores = sorted(relevance_scores, reverse=True)
                dcg = sum((2**rel - 1) / np.log2(i + 2) for i, rel in enumerate(relevance_scores))
                idcg = sum((2**rel - 1) / np.log2(i + 2) for i, rel in enumerate(ideal_scores))
                ndcg = dcg / idcg if idcg > 0 else 0
            else:
                ndcg = 0

            # Precision@k 계산
            precision_5 = sum(ps['is_relevant'] for ps in paper_scores[:5]) / 5 if len(paper_scores) >= 5 else 0
            precision_10 = sum(ps['is_relevant'] for ps in paper_scores[:10]) / 10 if len(paper_scores) >= 10 else 0

            # 평가 지표 추가
            mrr_scores.append(mrr)
            ndcg_scores.append(ndcg)
            precision_at_5.append(precision_5)
            precision_at_10.append(precision_10)

            # 상위 5개 논문 출력 - 처음 5개 채용공고만 출력
            job_global_idx = batch_idx * job_batch_size + job_idx
            if job_global_idx < 5:
                print(f"\n채용공고: {job['field']}")
                print("상위 5개 추천 논문:")
                for i, ps in enumerate(paper_scores[:5], 1):
                    print(f"{i}. {ps['paper']['title']} (유사도: {ps['similarity']:.4f}, 스케일링: {ps['scaled_similarity']:.4f}, 종합점수: {ps['relevance_score']:.4f}, {ps['relevance_str']})")

    # 평균 평가 지표 계산 (데이터가 있는 경우만)
    avg_mrr = sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0
    avg_ndcg = sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0
    avg_precision_5 = sum(precision_at_5) / len(precision_at_5) if precision_at_5 else 0
    avg_precision_10 = sum(precision_at_10) / len(precision_at_10) if precision_at_10 else 0

    print("\n=== 평가 결과 ===")
    print(f"모델 유형: {'Hi-BERT' if is_hibert else '일반 모델'}")
    print(f"MRR: {avg_mrr:.4f}")
    print(f"NDCG@10: {avg_ndcg:.4f}")
    print(f"Precision@5: {avg_precision_5:.4f}")
    print(f"Precision@10: {avg_precision_10:.4f}")

    print("=== 모델 검증 완료 ===")
    return {
        'mrr': avg_mrr,
        'ndcg': avg_ndcg,
        'precision@5': avg_precision_5,
        'precision@10': avg_precision_10,
        'is_hibert': is_hibert
    }

def main():
    """메인 함수 - Hi-BERT 모델 학습 및 검증 (최적화 버전)"""
    print("=== Hi-BERT 모델 학습 시작 (최적화 버전) ===")
    start_time = time.time()

    # 출력 디렉토리 설정
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 데이터셋 정보 저장 파일 경로
    dataset_info_path = f"{OUTPUT_DIR}/dataset_info.json"

    # GPU 메모리 정리
    gc.collect()
    torch.cuda.empty_cache()

    # 디바이스 설정
    print(f"Using device: {DEVICE}")
    if torch.cuda.is_available():
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB Total")
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

    # 병렬 데이터 로딩 설정
    print("데이터 로딩 최적화 설정 적용 중...")

    # 병렬 처리로 온톨로지 데이터 로드
    def load_ontologies_in_parallel():
        train_jobs = load_ontology(ONTOLOGY_TRAIN_DIR, max_samples=None)
        test_jobs = load_ontology(ONTOLOGY_TEST_DIR, max_samples=None)
        val_jobs = load_ontology(ONTOLOGY_VALID_DIR, max_samples=None)
        return train_jobs, test_jobs, val_jobs

    print("채용공고 온톨로지 데이터 병렬 로드 중...")
    # 멀티스레딩을 사용하여 온톨로지 파일 로드
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future = executor.submit(load_ontologies_in_parallel)
        train_job_fields, test_job_fields, validation_job_fields = future.result()

    print(f"Loaded job fields: Train: {len(train_job_fields)}, Test: {len(test_job_fields)}, Validation: {len(validation_job_fields)}")

    # 논문 데이터 로드
    print("논문 데이터 로드 중...")
    papers = load_papers(PAPER_TRAIN_PATH, max_samples=None)
    print(f"Loaded {len(papers)} papers")

    # 이전 학습의 데이터셋 크기 로드 (있는 경우)
    previous_dataset_sizes = {}
    if os.path.exists(dataset_info_path):
        try:
            with open(dataset_info_path, 'r') as f:
                previous_dataset_sizes = json.load(f)
            print(f"이전 학습 데이터셋 크기: 채용공고(Train) {previous_dataset_sizes.get('train_job_count', 0)}개, "
                  f"채용공고(Test) {previous_dataset_sizes.get('test_job_count', 0)}개, "
                  f"채용공고(Validation) {previous_dataset_sizes.get('validation_job_count', 0)}개, "
                  f"논문 {previous_dataset_sizes.get('paper_count', 0)}개")
        except Exception as e:
            print(f"이전 데이터셋 정보 로드 실패: {e}")
            previous_dataset_sizes = {}

    # 현재 데이터셋 크기
    current_dataset_sizes = {
        'train_job_count': len(train_job_fields),
        'test_job_count': len(test_job_fields),
        'validation_job_count': len(validation_job_fields),
        'paper_count': len(papers)
    }

    # 데이터셋 크기 변경 여부 확인
    dataset_changed = (
        previous_dataset_sizes.get('train_job_count', 0) != current_dataset_sizes['train_job_count'] or
        previous_dataset_sizes.get('test_job_count', 0) != current_dataset_sizes['test_job_count'] or
        previous_dataset_sizes.get('validation_job_count', 0) != current_dataset_sizes['validation_job_count'] or
        previous_dataset_sizes.get('paper_count', 0) != current_dataset_sizes['paper_count']
    )

    # 체크포인트 초기화 여부 결정
    reset_checkpoints = IS_RESET_CHECKPOINT
    if dataset_changed:
        print("\n주의: 데이터셋 크기가 변경되었습니다!")
        print(f"이전: 채용공고(Train) {previous_dataset_sizes.get('train_job_count', 0)}개, "
              f"채용공고(Test) {previous_dataset_sizes.get('test_job_count', 0)}개, "
              f"채용공고(Validation) {previous_dataset_sizes.get('validation_job_count', 0)}개, "
              f"논문 {previous_dataset_sizes.get('paper_count', 0)}개")
        print(f"현재: 채용공고(Train) {current_dataset_sizes['train_job_count']}개, "
              f"채용공고(Test) {current_dataset_sizes['test_job_count']}개, "
              f"채용공고(Validation) {current_dataset_sizes['validation_job_count']}개, "
              f"논문 {current_dataset_sizes['paper_count']}개")
        print("데이터셋 변경으로 인해 체크포인트를 초기화하고 처음부터 학습을 진행합니다.")
        reset_checkpoints = True  # 데이터셋 변경 시 강제로 체크포인트 초기화

        # 기존 체크포인트 디렉토리 백업
        mlm_checkpoint_dir = f"{OUTPUT_DIR}/mlm_checkpoints"
        cl_checkpoint_dir = f"{OUTPUT_DIR}/cl_checkpoints"
        if os.path.exists(mlm_checkpoint_dir):
            backup_dir = f"{mlm_checkpoint_dir}_backup_{int(time.time())}"
            os.rename(mlm_checkpoint_dir, backup_dir)
            print(f"MLM 체크포인트 백업 완료: {backup_dir}")

        if os.path.exists(cl_checkpoint_dir):
            backup_dir = f"{cl_checkpoint_dir}_backup_{int(time.time())}"
            os.rename(cl_checkpoint_dir, backup_dir)
            print(f"대조학습 체크포인트 백업 완료: {backup_dir}")

        # 기존 weak_supervision_pairs.pkl 파일도 백업
        pairs_file = f"{OUTPUT_DIR}/weak_supervision_pairs.pkl"
        if os.path.exists(pairs_file):
            backup_file = f"{pairs_file}.backup_{int(time.time())}"
            os.rename(pairs_file, backup_file)
            print(f"약한 감독 페어 파일 백업 완료: {backup_file}")
    else:
        print("데이터셋 크기가 이전과 동일합니다.")

    # 현재 데이터셋 크기 저장
    with open(dataset_info_path, 'w') as f:
        json.dump(current_dataset_sizes, f)

    # 1단계: 템플릿 기반 문장 생성 (병렬 처리)
    print("템플릿 기반 문장 생성 중...")
    # 템플릿 생성 전 데이터 샘플링 (메모리 절약)
    train_job_sample = random.sample(train_job_fields, min(500, len(train_job_fields)))
    paper_sample = random.sample(papers, min(1000, len(papers)))

    # 학습에는 train_job_fields만 사용
    template_sentences = generate_template_sentences(train_job_sample, paper_sample, templates)
    # 생성된 템플릿이 너무 많으면 샘플링
    if len(template_sentences) > MAX_TEMPLATE_COUNT:
        template_sentences = random.sample(template_sentences, MAX_TEMPLATE_COUNT)
    print(f"Generated {len(template_sentences)} template sentences")

    # 메모리 정리
    del train_job_sample
    del paper_sample
    gc.collect()
    torch.cuda.empty_cache()

    # 2단계: MLM 토크나이저 및 기본 모델 로드
    print("MLM 토크나이저 및 모델 로드 중...")
    mlm_tokenizer = BertTokenizer.from_pretrained(BASE_MODEL)

    # MLM 학습이 이미 완료되었는지 확인 (체크포인트 초기화를 고려)
    mlm_completed = os.path.exists(f"{OUTPUT_DIR}/mlm_best_model/pytorch_model.bin") and not reset_checkpoints

    if mlm_completed:
        print("MLM 학습 결과를 로드합니다...")
        # MLM 모델 로드
        mlm_model = BertForMaskedLM.from_pretrained(f"{OUTPUT_DIR}/mlm_best_model")
        mlm_tokenizer = BertTokenizer.from_pretrained(f"{OUTPUT_DIR}/mlm_best_model")
    else:
        # 3단계: 처음부터 MLM 학습 (학습에는 train_job_fields만 사용)
        base_model = BertForMaskedLM.from_pretrained(BASE_MODEL)
        mlm_model, mlm_tokenizer = train_domain_adapted_mlm(
            train_job_fields, papers, mlm_tokenizer, base_model, OUTPUT_DIR, template_sentences,
            reset_checkpoints=reset_checkpoints
        )

    # 메모리 정리
    if 'base_model' in locals():
        del base_model
    if 'mlm_model' in locals():
        del mlm_model
    gc.collect()
    torch.cuda.empty_cache()

    # 대조학습에도 동일한 BERT 모델 사용 (일관성 유지)
    print("대조학습을 위한 Hi-BERT 모델 로드 중...")

    # 기본 BERT 모델 로드 (MLM 모델에서 초기화)
    cl_model = BertModel.from_pretrained(BASE_MODEL)
    cl_tokenizer = mlm_tokenizer  # MLM과 동일한 토크나이저 사용

    # 대조학습이 이미 완료되었는지 확인 (체크포인트 초기화를 고려)
    cl_completed = os.path.exists(f"{OUTPUT_DIR}/cl_best_model.pt") and not reset_checkpoints

    if cl_completed:
        print("대조학습 결과를 로드합니다...")
        # Hi-BERT 이중 인코더 모델 초기화
        dual_encoder = DualEncoderModel(cl_model, projection_dim=PROJECTION_DIM)
        # 저장된 가중치 로드
        checkpoint = torch.load(f"{OUTPUT_DIR}/cl_best_model.pt", map_location=DEVICE)
        dual_encoder.load_state_dict(checkpoint['model_state_dict'])
        # 모델을 명시적으로 디바이스로 이동
        dual_encoder = dual_encoder.to(DEVICE)
    else:
        # 4단계: 대조학습 기반 Hi-BERT 모델 학습 (최적화된 함수 사용)
        dual_encoder = train_contrastive_learning(
            train_job_fields, papers, cl_model, cl_tokenizer, OUTPUT_DIR, template_sentences,
            reset_checkpoints=reset_checkpoints
        )

    # 5단계: 테스트 세트로 모델 검증 (최적화된 검증 함수 사용)
    print("\n=== 테스트 세트에서 Hi-BERT 모델 검증 ===")
    test_results = validate_model(
        dual_encoder, cl_tokenizer, test_job_fields, papers, DEVICE
    )

    # 6단계: 검증 세트로 모델 검증 (최적화된 검증 함수 사용)
    print("\n=== 검증 세트에서 Hi-BERT 모델 검증 ===")
    validation_results = validate_model(
        dual_encoder, cl_tokenizer, validation_job_fields, papers, DEVICE
    )

    # 최종 결과
    print("\n=== Hi-BERT 학습 결과 ===")
    print("테스트 세트 결과:")
    print(f"MRR: {test_results['mrr']:.4f}")
    print(f"NDCG@10: {test_results['ndcg']:.4f}")
    print(f"Precision@5: {test_results['precision@5']:.4f}")
    print(f"Precision@10: {test_results['precision@10']:.4f}")

    print("\n검증 세트 결과:")
    print(f"MRR: {validation_results['mrr']:.4f}")
    print(f"NDCG@10: {validation_results['ndcg']:.4f}")
    print(f"Precision@5: {validation_results['precision@5']:.4f}")
    print(f"Precision@10: {validation_results['precision@10']:.4f}")

    # 총 소요 시간 출력
    total_time = time.time() - start_time
    print(f"\n총 학습 소요 시간: {total_time:.2f}초 ({total_time/60:.2f}분)")
    print(f"모든 모델이 {OUTPUT_DIR}에 저장되었습니다.")
    print("=== Hi-BERT 모델 학습 완료 ===")

# 최종 실행 코드
if __name__ == "__main__":
    # 모듈 임포트 확인
    required_modules = {
        'torch': torch,
        'numpy': np,
        'pandas': pd,
        'transformers': None,
        'tqdm': tqdm,
        'concurrent.futures': concurrent.futures
    }

    # 모듈 버전 출력
    print("=== 모듈 버전 확인 ===")
    for module_name, module in required_modules.items():
        if module:
            if hasattr(module, '__version__'):
                print(f"{module_name}: {module.__version__}")
            else:
                print(f"{module_name}: 설치됨 (버전 정보 없음)")
        else:
            print(f"{module_name}: 설치됨")

    # GPU 정보 확인
    print("\n=== GPU 정보 확인 ===")
    if torch.cuda.is_available():
        print(f"사용 가능한 GPU: {torch.cuda.device_count()}개")
        print(f"현재 GPU: {torch.cuda.current_device()} - {torch.cuda.get_device_name(0)}")
        print(f"CUDA 버전: {torch.version.cuda}")
        print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("GPU를 사용할 수 없습니다. CPU 모드로 실행합니다.")

    # 경로 확인
    print("\n=== 경로 확인 ===")
    paths = {
        "온톨로지(훈련)": ONTOLOGY_TRAIN_DIR,
        "온톨로지(테스트)": ONTOLOGY_TEST_DIR,
        "온톨로지(검증)": ONTOLOGY_VALID_DIR,
        "논문 데이터": PAPER_TRAIN_PATH,
        "모델 출력": OUTPUT_DIR,
        "불용어": STOPWORDS_PATH
    }

    for name, path in paths.items():
        exists = os.path.exists(path)
        status = "존재함" if exists else "존재하지 않음"
        print(f"{name}: {path} ({status})")

    # 주요 설정 확인
    print("\n=== 주요 설정 확인 ===")
    print(f"기본 모델: {BASE_MODEL}")
    print(f"최대 길이: {MAX_LENGTH}")
    print(f"배치 크기: {BATCH_SIZE}")
    print(f"MLM 학습률: {MLM_LEARNING_RATE}")
    print(f"대조학습 학습률: {CL_LEARNING_RATE}")
    print(f"MLM 에포크: {MLM_EPOCHS}")
    print(f"대조학습 에포크: {CL_EPOCHS}")
    print(f"그라디언트 누적 단계: {ACCUMULATION_STEPS}")
    print(f"체크포인트 초기화: {IS_RESET_CHECKPOINT}")

    # 메인 함수 실행
    try:
        print("\n=== Hi-BERT 모델 학습 시작 ===")
        main()
    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback
        traceback.print_exc()
        print("\n프로그램이 오류로 종료되었습니다.")

Using device: cuda
Loaded 10921 stopwords from /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/datasets/recruitment_stopwords2.csv
=== 모듈 버전 확인 ===
torch: 2.6.0+cu124
numpy: 2.0.2
pandas: 2.2.2
transformers: 설치됨
tqdm: 설치됨 (버전 정보 없음)
concurrent.futures: 설치됨 (버전 정보 없음)

=== GPU 정보 확인 ===
사용 가능한 GPU: 1개
현재 GPU: 0 - NVIDIA A100-SXM4-40GB
CUDA 버전: 12.4
GPU 메모리: 42.47 GB

=== 경로 확인 ===
온톨로지(훈련): /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/training (존재함)
온톨로지(테스트): /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/test (존재함)
온톨로지(검증): /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/validation (존재함)
논문 데이터: /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/datasets/combined_articles_train.csv (존재함)
모델 출력: /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/models/hi_bert_model_kosimcse (존재함)
불용어: /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/datasets/recruitment_stopwords2.csv (존재함)

=== 주요 설정 확인 ===
기본 모델: BM-K/KoSimCSE-bert
최대

Loading ontology files: 100%|██████████| 1000/1000 [00:07<00:00, 139.64it/s]



처리된 첫 번째 채용공고 정보:
ID: http://recruitontology.org/Recruit_recruitments_867
분야: 양자광학1
키워드: ['양자광', '양자회로', '최적화', '정밀측정', '양자센싱기술']
Successfully loaded 1000 job fields from 1000 files
Found 277 ontology files in /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/test
디버깅: 첫 번째 파일 경로 = /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/test/recruitments_1070-jsonld.json
파일 내용 (처음 500자):
{
  "@context": {
    "@vocab": "http://recruitontology.org/",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "http://recruitontology.org/Recruit_recruitments_1070",
  "@type": "Recruitment",
  "hasRecruitId": "recruitments_1070",
  "hasPdfUrl": "",
  "hasJobPosting": {
    "@type": "JobPosting",
    "hasOrganization": "호서대학교",
    "hasLocation": "충청남도 아산시",
    "hasTitle": "호서대학교 해양IT융합기술연구소 연구원, 연구보조원 채용"
  },
  "hasJobFields": [
    {
      "@type": "JobField",
      "hasDepartm
JSON 구조:
주요 키: ['@context', '@id', '@type', 'hasRecruitId', 'hasPdfUrl', 'hasJobPosti

Loading ontology files: 100%|██████████| 277/277 [00:00<00:00, 359.89it/s]



처리된 첫 번째 채용공고 정보:
ID: http://recruitontology.org/Recruit_recruitments_1070
분야: 연구원, 연구보조원
키워드: ['통신 알고리즘', '하드웨어 설계', '수중통신', 'FPGA', 'DSP', 'C/C++', 'Matlab']
Successfully loaded 277 job fields from 277 files
Found 279 ontology files in /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/validation
디버깅: 첫 번째 파일 경로 = /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/ontologies/validation/recruitments_1433-jsonld.json
파일 내용 (처음 500자):
{
  "@context": {
    "@vocab": "http://recruitontology.org/",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "http://recruitontology.org/Recruit_recruitments_1433",
  "@type": "Recruitment",
  "hasRecruitId": "recruitments_1433",
  "hasPdfUrl": "",
  "hasJobPosting": {
    "@type": "JobPosting",
    "hasOrganization": "유원대학교",
    "hasLocation": "아산",
    "hasTitle": "유원대학교 2021학년도 전임교원 초빙 공고(4차)"
  },
  "hasJobFields": [
    {
      "@type": "JobField",
      "hasDepartment": "아산
JSON 구조:
주요 키: ['@context', '@id', '@type', '

...

논문 임베딩 100개 사전 계산 완료


채용공고 배치 평가:  40%|████      | 2/5 [00:00<00:00, 16.17it/s]


채용공고: 정보통신 및 보안(정보통신,해킹보안,컴퓨터보안,네트워크보안,멀티미디어보안,사물인터넷,인공지능 등)
상위 5개 추천 논문:
1. AI 시스템의 위험 완화를 위한 정책적 접근방안 연구: AI 영향평가를 중심으로 (유사도: 0.1108, 스케일링: 0.5542, 종합점수: 0.6662, 관련 (유사도 0.1108, 키워드 일치 1개 (인공지능), 부분 일치 1개))
2. 군사용 인공지능의 한미간 개발체계 비교 (유사도: 0.0795, 스케일링: 0.3974, 종합점수: 0.6192, 관련 (유사도 0.0795, 키워드 일치 1개 (인공지능), 부분 일치 1개))
3. 국방 분야에서의 LLM 활용 문제점과 해결 전략 (유사도: 0.1313, 스케일링: 0.6564, 종합점수: 0.4969, 관련 (유사도 0.1313, 키워드 일치 1개 (인공지능)))
4. 대학 교양 교육을 위한 AI·소프트웨어 교육 (유사도: 0.0661, 스케일링: 0.3303, 종합점수: 0.3991, 관련 (유사도 0.0661, 키워드 일치 1개 (인공지능)))
5. 메타버스(Metaverse) 대중화 시대의 기독교윤리적 책임 (유사도: 0.0363, 스케일링: 0.1814, 종합점수: 0.3544, 관련 (키워드 일치 1개 (인공지능)))

채용공고: IT 전 분야
상위 5개 추천 논문:
1. 한국 60세 이상 노인에서 사구체여과율과 악력의 연관성: 2016–2018년 국민건강영양조사 (유사도: 0.1357, 스케일링: 0.6787, 종합점수: 0.2036, 관련 (유사도 0.1357))
2. 현장시험을 통한 항만 구역 내 블록 포장의 침하 특성 분석 (유사도: 0.1318, 스케일링: 0.6590, 종합점수: 0.1977, 관련 (유사도 0.1318))
3. 이중막 구조를 적용한 우주용 전개형 메쉬 안테나의 열적 특성 분석 (유사도: 0.1315, 스케일링: 0.6573, 종합점수: 0.1972, 관련 (유사도 0.1315))
4. 초기 국어문법론 탐구의 터전이 된 『진단학보』

채용공고 배치 평가: 100%|██████████| 5/5 [00:00<00:00, 19.19it/s]


=== 평가 결과 ===
모델 유형: Hi-BERT
MRR: 0.9750
NDCG@10: 0.9836
Precision@5: 0.9800
Precision@10: 0.9400
=== 모델 검증 완료 ===

=== Hi-BERT 학습 결과 ===
테스트 세트 결과:
MRR: 0.9750
NDCG@10: 0.9633
Precision@5: 0.9000
Precision@10: 0.9250

검증 세트 결과:
MRR: 0.9750
NDCG@10: 0.9836
Precision@5: 0.9800
Precision@10: 0.9400

총 학습 소요 시간: 1080.45초 (18.01분)
모든 모델이 /content/drive/MyDrive/Workspaces/Hibrainnet/Hi-BERT/models/hi_bert_model_kosimcse에 저장되었습니다.
=== Hi-BERT 모델 학습 완료 ===
